<a href="https://colab.research.google.com/github/J-Nobull/Migration_Research/blob/main/Migration_Capstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Capstone Research Project on Migration Within the USA

In [ ]:
!pip install census

In [ ]:
# Setup initial environment
import pandas as pd
import numpy as np
from census import Census
import os
import requests
import time
from getpass import getpass
#from us import states

print('Environment Ready')

Environment Ready


#Define Keys

In [ ]:
def get_api_key(name):
    key = os.getenv(name)
    if not key:
        key = getpass(f"Enter {name} (hidden input): ")
    return key

API_KEY_BEA = get_api_key('API_KEY_BEA')
#API_KEY_BLS = get_api_key('API_KEY_BLS')
API_KEY_CENSUS = get_api_key('API_KEY_CENSUS')
#API_KEY_IRS = get_api_key('API_KEY_IRS')
#API_KEY_USDA = get_api_key('API_KEY_USDA')

print('API keys loaded')

#Import from Bureau of Economic Analysis (1 of 5)

In [ ]:
API_KEY_BEA = 'key_here'
BEA_URL = 'https://apps.bea.gov/api/data'
YEARS = ','.join([str(y) for y in range(2011, 2022)])

# Define Tables with LineCodes and GeoFips
TABLES = [
    {'name': 'PARPP', 'linecode': '3', 'geofips': 'PORT', 'desc': 'rpp_portions'}, # Cost of living for Metro/Non-metro
    {'name': 'MARPP', 'linecode': '3', 'geofips': 'MSA', 'desc': 'rpp_msa'}, # Cost of living for Urban areas (MSAs)
    {'name': 'CAINC1', 'linecode': '3', 'geofips': 'COUNTY', 'desc': 'percapita_income'}, # County Income data
    {'name': 'CAGDP1', 'linecode': '1', 'geofips': 'COUNTY', 'desc': 'real_gdp'}] # County GDP

# Fetch tables
print('Downloading BEA data (2011-2021)...\n')

for table in TABLES:

    print(f'Fetching {table["desc"]} ({table["name"]})...')

    params = {
        'UserID': API_KEY_BEA,
        'method': 'GetData',
        'datasetname': 'Regional',
        'TableName': table['name'],
        'LineCode': table['linecode'],
        'Year': YEARS,
        'GeoFips': table['geofips'],
        'ResultFormat': 'json'}

    response = requests.get(BEA_URL, params=params, timeout=120)
    data = response.json()

    # Show errors
    if 'Error' in data.get('BEAAPI', {}):
        print(f'❌ Error: {data["BEAAPI"]["Error"]["Detail"]}')
        continue

    # Save to CSV
    BEA_df = pd.DataFrame(data['BEAAPI']['Results']['Data'])
    filename = f'data_BEA_{table["desc"]}.csv'
    BEA_df.to_csv(BEA_import, index=False)

    print(f'Saved {len(df):,} rows to {BEA_import}')
    time.sleep(2)
print('\nBEA retrieval complete!')

# Import from Census Bureau (2 of 5)

In [ ]:
CENSUS_API_KEY = '5f7c8b0ecd185273567de16a4c09273088d4340b'
Census_URL = 'https://api.census.gov/data'
YEARS = list(range(2011, 2022))

# Final variable list
ACS_VARS = {
    # Population
    'B01003_001E': 'total_population',

    # Age
    'B01002_001E': 'median_age',

    # Housing
    'B25003_001E': 'housing_total',
    'B25003_002E': 'owner_occupied',
    'B25003_003E': 'renter_occupied',
    'B25002_003E': 'vacant',
    'B25077_001E': 'median_home_value',

    # Households
    'B11001_002E': 'family_households',

    # Marital Status
    'B12001_001E': 'marital_total',
    'B12001_003E': 'never_married_male',
    'B12001_004E': 'now_married_male',
    'B12001_009E': 'widowed_male',
    'B12001_010E': 'divorced_male',
    'B12001_012E': 'never_married_female',
    'B12001_013E': 'now_married_female',
    'B12001_018E': 'widowed_female',
    'B12001_019E': 'divorced_female',

    # Children (all under 18)
    'B09001_002E': 'under_18_in_hh',

    # Race/Ethnicity (sums to total)
    'B03002_003E': 'white',
    'B03002_004E': 'black',
    'B03002_005E': 'native',
    'B03002_006E': 'asian',
    'B03002_007E': 'pacific_islander',
    'B03002_008E': 'other_race',
    'B03002_009E': 'two_or_more_nh',
    'B03002_012E': 'hispanic',

    # Education
    'B15002_001E': 'education_total_sex',
    'B15002_011E': 'male_complete_hs',
    'B15002_012E': 'male_less1yr_college',
    'B15002_013E': 'male_more1yr_college',
    'B15002_014E': 'male_associates',
    'B15002_015E': 'male_bachelors',
    'B15002_016E': 'male_masters',
    'B15002_017E': 'male_professional',
    'B15002_018E': 'male_doctorate',
    'B15002_028E': 'female_complete_hs',
    'B15002_029E': 'female_less1yr_college',
    'B15002_030E': 'female_more1yr_college',
    'B15002_031E': 'female_associates',
    'B15002_032E': 'female_bachelors',
    'B15002_033E': 'female_masters',
    'B15002_034E': 'female_professional',
    'B15002_035E': 'female_doctorate',

    # Income
    'B19013_001E': 'median_hh_income',

    # Employment
    'B23025_004E': 'employed',
    'B23025_005E': 'unemployed',

    # Commute Time (all categories)
    'B08303_002E': 'commute_less_5min',
    'B08303_003E': 'commute_5_9min',
    'B08303_004E': 'commute_10_14min',
    'B08303_005E': 'commute_15_19min',
    'B08303_006E': 'commute_20_24min',
    'B08303_007E': 'commute_25_29min',
    'B08303_008E': 'commute_30_34min',
    'B08303_009E': 'commute_35_39min',
    'B08303_010E': 'commute_40_44min',
    'B08303_011E': 'commute_45_59min',
    'B08303_012E': 'commute_60_89min',
    'B08303_013E': 'commute_90_plus_min',

    # Worked from home
    'B08137_020E': 'work_in owned_home',
    'B08137_021E': 'work_in_rental',

    # Estate taxes paid
    'B25103_002E': 'Median_property_taxes',

    # Industry
    'C24060_001E': 'occupation_total',
    'C24060_002E': 'Mgmt_biz_sci_arts',
    'C24060_003E': 'Services',
    'C24060_004E': 'Sales_admin',
    'C24060_005E': 'Nat-rsrc_constr_maint',
    'C24060_006E': 'Prod_transp_mvng'}

def fetch_acs_county_data_batch(year, variables, api_key):
    '''Fetch ACS 5-Year data for a given year and list of variables.'''

    print(f'  Fetching ACS {year} 5-Year estimates for {len(variables)} variables...')

    # Only include the variable IDs in the 'get' parameter
    var_list = ','.join(variables)

    params = {
        'get': var_list,
        'for': 'county:*',
        'key': api_key}

    url = f'{Census_URL}/{year}/acs/acs5'

    try:
        response = requests.get(url, params=params, timeout=120)
        response.raise_for_status()

        data = response.json()
        if len(data) > 1:
            # Census API typically returns state and county FIPS as the last two columns
            # data has headers
            df = pd.DataFrame(data[1:], columns=data[0])
            print(f'    ✓ Successfully fetched {len(df):,} counties')
            return df
        else:
            print(f"    No data returned for year {year} with variables {variables}")
            return pd.DataFrame() # Return empty DataFrame if no data rows

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for year {year}: {e}")
        if response is not None:
            print(f"Status Code: {response.status_code}")
            print(f"Response Text: {response.text}")
        return pd.DataFrame()

# Fetch data for all years, splitting variables into batches
print(f'Downloading Census ACS 5-Year data for {YEARS[0]}-{YEARS[-1]} in batches...\n')

all_data_dfs = []
variable_list = list(ACS_VARS.keys())
# Assuming the total number of variables now requires two batches (e.g., > 50)
# Splitting into two batches for demonstration, adjust batch_size if needed
batch1_vars = variable_list[:45]
batch2_vars = variable_list[45:]

for year in YEARS:
    year_data_dfs = []
    print(f'Processing year {year}...')

    # Fetch batch 1
    if batch1_vars:
      df_batch1 = fetch_acs_county_data_batch(year, batch1_vars, CENSUS_API_KEY)
      if not df_batch1.empty:
          year_data_dfs.append(df_batch1)
      time.sleep(0.5) # Small delay

    # Fetch batch 2
    if batch2_vars:
      df_batch2 = fetch_acs_county_data_batch(year, batch2_vars, CENSUS_API_KEY)
      # Ensure columns match for merging if some variables are missing in earlier years
      # For simplicity here, assuming consistent columns across batches for a given year
      if not df_batch2.empty:
           year_data_dfs.append(df_batch2)
      time.sleep(0.5) # Small delay

    # Merge dataframes for the current year
    if year_data_dfs:
        # The first dataframe includes state and county
        merged_year_df = year_data_dfs[0]
        for j in range(1, len(year_data_dfs)):
            # Merge subsequent dataframes on the identifier columns
            # Assuming 'state' and 'county' are always returned
            merged_year_df = pd.merge(merged_year_df, year_data_dfs[j], on=['state', 'county'], how='outer')

        # Add Year after merging batches for the year
        merged_year_df['Year'] = year

        all_data_dfs.append(merged_year_df)
        print(f'  ✓ Finished processing year {year} with {len(merged_year_df):,} rows')
    else:
        print(f'  ✗ No data collected for year {year}')

# Combine all years into CEN_df
if all_data_dfs:
    CEN_df = pd.concat(all_data_dfs, ignore_index=True)

    # Apply renaming after combining all years
    rename_dict = {k: v for k, v in ACS_VARS.items()}
    # Filter rename_dict to only include columns actually present in the combined DataFrame before renaming
    present_rename_dict = {k: v for k, v in rename_dict.items() if k in CEN_df.columns}
    CEN_df = CEN_df.rename(columns=present_rename_dict)


    # Convert all numeric columns based on the *original* column names (variable IDs)
    # before renaming, or based on the *renamed* columns after renaming.
    # Since we've renamed, let's iterate through the *renamed* columns we expect
    # to ensure they are numeric if present.
    for col in ACS_VARS.values():
        if col in CEN_df.columns:
            # Handle potential non-numeric values resulting from merges by coercing errors
            CEN_df[col] = pd.to_numeric(CEN_df[col], errors='coerce')


    # Add FIPS column with leading zeros
    if 'state' in CEN_df.columns and 'county' in CEN_df.columns:
        CEN_df['FIPS'] = CEN_df['state'].astype(str).str.zfill(2) + CEN_df['county'].astype(str).str.zfill(3)

    # Ensure Year is integer type
    if 'Year' in CEN_df.columns: # Check if 'Year' column exists before converting
      CEN_df['Year'] = CEN_df['Year'].astype(int)

    # Keep only FIPS, Year, state, county, and all variables (renamed)
    # Preserve the order of columns as they are after renaming and adding FIPS/Year.
    # The order will be the order returned by the API for batch 1, then batch 2,
    # followed by state, county, FIPS, and Year.
    # To strictly preserve the user's preferred order, explicit reordering would be needed,
    # but the user requested not to touch the variables/order beyond fixing issues.
    # The current order adds FIPS and Year at the end by default. Let's keep the
    # state and county columns as they are needed for merging with other datasets.
    final_cols_order = [col for col in CEN_df.columns if col not in ['FIPS', 'Year']]
    # Insert FIPS and Year at the beginning or keep them at the end based on typical use.
    # Let's keep them at the beginning for ease of use as identifiers.
    final_cols_order = ['FIPS', 'Year'] + [col for col in final_cols_order if col not in ['state', 'county']] + ['state', 'county']
    CEN_df = CEN_df[final_cols_order].copy()


    # Save to CSV
    CEN_df.to_csv('Census_data_2011_2021.csv', index=False) # Updated filename

    print(f'\n✓ Saved {len(CEN_df):,} rows to Census_data_2011_2021.csv')
    print(f'  Counties: {CEN_df["FIPS"].nunique()}')
    print(f'  Years: {CEN_df["Year"].min()}-{CEN_df["Year"].max()}')
    # Count variables based on the number of columns in the final dataframe excluding identifiers
    print(f'  Variables: {len(CEN_df.columns) - 4}') # Subtract FIPS, Year, state, county

else:
    print("\n✗ No data was successfully downloaded for any year.")


Processing year 2011...
  Fetching ACS 2011 5-Year estimates for 45 variables...
    ✓ Successfully fetched 3,221 counties
  Fetching ACS 2011 5-Year estimates for 22 variables...
    ✓ Successfully fetched 3,221 counties
  ✓ Finished processing year 2011 with 3,221 rows
Processing year 2012...
  Fetching ACS 2012 5-Year estimates for 45 variables...
    ✓ Successfully fetched 3,221 counties
  Fetching ACS 2012 5-Year estimates for 22 variables...
    ✓ Successfully fetched 3,221 counties
  ✓ Finished processing year 2012 with 3,221 rows
Processing year 2013...
  Fetching ACS 2013 5-Year estimates for 45 variables...
    ✓ Successfully fetched 3,221 counties
  Fetching ACS 2013 5-Year estimates for 22 variables...
    ✓ Successfully fetched 3,221 counties
  ✓ Finished processing year 2013 with 3,221 rows
Processing year 2014...
  Fetching ACS 2014 5-Year estimates for 45 variables...
    ✓ Successfully fetched 3,220 counties
  Fetching ACS 2014 5-Year estimates for 22 variables...
   

In [ ]:

# Fetch data  in batches
print(f'Downloading Census ACS 5-Year data for {YEARS[0]} in batches...\n')

all_variables = list(ACS_VARS.keys())
# Split variables into two batches (since we are over 50)
batch1_vars = all_variables[:50]
batch2_vars = all_variables[50:]

year = YEARS
batch_dfs = []

# Fetch batch 1
if batch1_vars:
  df_batch1 = fetch_acs_county_data_batch(year, batch1_vars, CENSUS_API_KEY)
  if not df_batch1.empty:
      batch_dfs.append(df_batch1)
  time.sleep(0.5) # Small delay

# Fetch batch 2
if batch2_vars:
  df_batch2 = fetch_acs_county_data_batch(year, batch2_vars, CENSUS_API_KEY)
  if not df_batch2.empty:
      batch_dfs.append(df_batch2)
  time.sleep(0.5) # Small delay


# Merge the dataframes from all batches
if batch_dfs:
    # The first dataframe includes state and county
    merged_df = batch_dfs[0]
    for j in range(1, len(batch_dfs)):
        # Merge subsequent dataframes on the identifier columns
        # Assuming 'state' and 'county' are always returned
        merged_df = pd.merge(merged_df, batch_dfs[j], on=['state', 'county'], how='outer')

    print('\ndata download complete. Merged batches.')

    # Apply renaming after combining batches
    rename_dict = {k: v for k, v in ACS_VARS.items()}
    merged_df = merged_df.rename(columns=rename_dict)

    # Convert all numeric columns based on the renamed columns
    # Iterate through the *renamed* columns that are expected to be numeric
    for col in ACS_VARS.values():
        if col in merged_df.columns:
            # Handle potential non-numeric values resulting from merges by coercing errors
            merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')

    # Add FIPS column with leading zeros
    if 'state' in merged_df.columns and 'county' in merged_df.columns:
        merged_df['FIPS'] = merged_df['state'].astype(str).str.zfill(2) + merged_df['county'].astype(str).str.zfill(3)

    # Add Year column and ensure it's integer type
    merged_df['Year'] = year
    merged_df['Year'] = merged_df['Year'].astype(int)

    # Keep only FIPS, Year, and all variables (renamed), ensuring columns exist
    # Preserve the order of variables as they were returned by the API + FIPS and Year
    # The order will be FIPS, Year, followed by the variables in the order they were returned by the API
    # and then renamed.
    ordered_cols = ['FIPS', 'Year'] + [col for col in merged_df.columns if col not in ['FIPS', 'Year', 'state', 'county']]
    census_df = merged_df[ordered_cols].copy()


    # Save to CSV
    census_df.to_csv('Census_import.csv', index=False) # Updated filename

    print(f'\n✓ Saved {len(census_df):,} rows to Census_import.csv')
    print(f'  Counties: {census_df["FIPS"].nunique()}')
    print(f'  Year: {census_df["Year"].iloc[0]}')
    print(f'  Variables: {len([col for col in ACS_VARS.values() if col in census_df.columns])}') # Count variables actually in the DF

else:
    print("\n Data download failed.")



  Fetching ACS 2011 5-Year estimates for 50 variables...
    ✓ Successfully fetched 3,221 counties
  Fetching ACS 2011 5-Year estimates for 5 variables...
    ✓ Successfully fetched 3,221 counties

2011 data download complete. Merged batches.

✓ Saved 3,221 rows to Census_2011.csv
  Counties: 3221
  Year: 2011
  Variables: 55


In [ ]:
def fetch_acs_county_data(year, variables, api_key):

    print(f' Fetching ACS {year} 5-Year estimates for {len(variables)} variables...')

    # Only include the variable IDs in the 'get' parameter
    var_list = ','.join(variables)

    params = {
        'get': var_list,
        'for': 'county:*',
        'key': api_key
    }

    url = f'{Census_URL}/{year}/acs/acs5'

    try:
        response = requests.get(url, params=params, timeout=120)
        response.raise_for_status()

        data = response.json()
        if len(data) > 1:
            # Census API typically returns state and county FIPS as the last two columns
            # even if not explicitly requested in 'get'.
            # The first row is headers, subsequent rows are data
            df = pd.DataFrame(data[1:], columns=data[0])
            print(f'    ✓ Successfully fetched {len(df):,} counties')
            return df
        else:
            print(f"    No data returned for year {year} with variables {variables}")
            return pd.DataFrame() # Return empty DataFrame if no data rows

    except requests.exceptions.RequestException as e:
        print(f"  Error fetching data for year {year}: {e}")
        if response is not None:
            print(f"  Status Code: {response.status_code}")
            print(f"  Response Text: {response.text}")
        return pd.DataFrame()

print(f'Downloading Census ACS 5-Year data for {YEARS[0]}-{YEARS[-1]}...\n')

all_data_dfs = []
variable_list = list(ACS_VARS.keys())

for year in YEARS:
    print(f'Processing year {year}...')
    df = fetch_acs_county_data(year, variable_list, CENSUS_API_KEY) # Pass all variables
    if not df.empty:
        # Add the Year column to the DataFrame for the current year
        df['Year'] = year # Add year here
        all_data_dfs.append(df)
    time.sleep(1) # Small delay between year requests

# Combine all years into CEN_df
if all_data_dfs:
    CEN_df = pd.concat(all_data_dfs, ignore_index=True)

    # Apply renaming after combining all years
    rename_dict = {k: v for k, v in ACS_VARS.items()}
    CEN_df = CEN_df.rename(columns=rename_dict)

    # Convert all numeric columns based on the renamed columns
    for col in ACS_VARS.values():
        if col in CEN_df.columns:
            # Handle potential non-numeric values resulting from merges by coercing errors
            CEN_df[col] = pd.to_numeric(CEN_df[col], errors='coerce')

    # Add FIPS column with leading zeros
    if 'state' in CEN_df.columns and 'county' in CEN_df.columns:
        CEN_df['FIPS'] = CEN_df['state'].astype(str).str.zfill(2) + CEN_df['county'].astype(str).str.zfill(3)

    # Ensure Year is integer type
    if 'Year' in CEN_df.columns: # Check if 'Year' column exists before converting
      CEN_df['Year'] = CEN_df['Year'].astype(int)

    # Keep only FIPS, Year, and all variables (renamed), ensuring columns exist and preserving original order
    # We will keep the original order of columns returned by the API for each year's batch,
    # plus the 'FIPS' and 'Year' columns.
    # The order will be the order of columns in the first downloaded dataframe,
    # with 'FIPS' and 'Year' potentially added at the end unless explicitly reordered later.
    # To preserve the user's desired final order, explicit column reordering would be needed,
    # but the user requested not to change the variable order.
    # The current approach adds FIPS and Year at the end.


    # Save to CSV
    CEN_df.to_csv('Census_data_2012_2021.csv', index=False) # Updated filename

    print(f'\n✓ Saved {len(CEN_df):,} rows to data_Census_ACS_2011_2021.csv')
    print(f'  Counties: {CEN_df["FIPS"].nunique()}')
    print(f'  Years: {CEN_df["Year"].min()}-{CEN_df["Year"].max()}')
    print(f'  Variables: {len(ACS_VARS)}')

else:
    print("\n✗ No data was successfully downloaded for any year.")

In [ ]:
# Get API key: https://api.census.gov/data/key_signup.html
key = Census('5f7c8b0ecd185273567de16a4c09273088d4340b')

# Import and clean data files  




In [ ]:
# Import the Census library and initialize the Census object
from census import Census
import pandas as pd
import requests # Import requests to check variable availability

# Get API key from the defined variable (assuming API_KEY_CENSUS or similar is available)
# Using the key defined in cell t1GjV13ndXCL directly as it's present in the notebook
c = Census('5f7c8b0ecd185273567de16a4c09273088d4340b')

# Base URL for Census API
BASE_URL = 'https://api.census.gov/data'


# Function to check if a variable is available for a given year and dataset
def is_variable_available(year, dataset, variable_id):
    variables_url = f'{BASE_URL}/{year}/{dataset}/variables.json'
    try:
        variables_response = requests.get(variables_url, timeout=120)
        variables_response.raise_for_status()
        available_vars_data = variables_response.json()
        return variable_id in available_vars_data.get('variables', {})
    except requests.exceptions.RequestException as e:
        print(f"Error checking variable availability for year {year}: {e}")
        return False


# Function to get property tax for one year
def get_property_tax(year):
    variable_id = 'B25103_002E' # Median property tax
    dataset = 'acs/acs5' # Dataset name for ACS 5-Year estimates

    # Check if the variable is available for this year
    if not is_variable_available(year, dataset, variable_id):
        print(f"  Variable {variable_id} not available for year {year}. Skipping.")
        return pd.DataFrame() # Return empty DataFrame if variable is not available

    print(f"  Fetching property tax data for year {year}...")
    data = c.acs5.state_county(
        fields=('NAME', variable_id),  # Median property tax
        state_fips='*',
        county_fips='*',
        year=year
    )

    if data: # Check if data was returned
        df = pd.DataFrame(data)
        df['Year'] = year
        # Ensure state and county FIPS are treated as strings with leading zeros before concatenation
        df['County_FIPS'] = df['state'].astype(str).str.zfill(2) + df['county'].astype(str).str.zfill(3)
        df = df.rename(columns={variable_id: 'Median_Property_Tax'})
        # Select and reorder columns to match desired output
        return df[['County_FIPS', 'NAME', 'Year', 'Median_Property_Tax']]
    else:
        print(f"  No data returned from API for year {year} even though variable was listed as available.")
        return pd.DataFrame()


# Get data for all years
print('Downloading property tax data (2011-2021)...')
dfs = [get_property_tax(year) for year in range(2011, 2022)]

# Filter out empty DataFrames before concatenation
dfs = [df for df in dfs if not df.empty]

property_tax_data = pd.concat(dfs, ignore_index=True)

# Save
property_tax_data.to_csv('property_taxes_2011_2021.csv', index=False)

# Confirm
print('\nProperty Tax Data Head:')
print(property_tax_data.head())
print('\nProperty Tax Data Tail:')
print(property_tax_data.tail())
print(f'\nTotal rows saved: {len(property_tax_data)}')

  Fetching property tax data for year 2011...
  Fetching property tax data for year 2012...
  Fetching property tax data for year 2013...
  Fetching property tax data for year 2014...
  Fetching property tax data for year 2015...
  Fetching property tax data for year 2016...
  Fetching property tax data for year 2017...
  Fetching property tax data for year 2018...
  Fetching property tax data for year 2019...
  Fetching property tax data for year 2020...
  Fetching property tax data for year 2021...

Property Tax Data Head:
  County_FIPS                               NAME  Year  Median_Property_Tax
0       37043        Clay County, North Carolina  2011                667.0
1       37051  Cumberland County, North Carolina  2011               1443.0
2       37081    Guilford County, North Carolina  2011               1644.0
3       37099     Jackson County, North Carolina  2011                708.0
4       37139  Pasquotank County, North Carolina  2011               1266.0

Property Tax

# Import DP1 data (1 of 3)
Demographic and Housing Characteristic (DHC) data from U.S. Census Bureau Decennial Survey at the County level. Includes 50 states, Puerto Rico, and D.C. *(Note on Alaska: See accompanying 'Alaska County' amalgamation file on github for method used to match census area to state senate district. DHC datafile combines 30 Alaskan census areas into 14 'County_fips' created for this analysis)*. Will drop Puerto Rico (rows 3144-3223). Also dropped Kalawao County, Hawaii: 82 residents, none of them voted, dropping will align it with MEDSL file when Kalawao county_fips (15005) is cleaned from MEDSL data.  
https://data.census.gov/table?q=DP1&g=010XX00US$0500000&y=2020


In [ ]:
# Import first dataset
DHC_import = pd.read_csv(
    'DECENNIALDP2020.DP1-AKfix.csv', header=1)

# Inspect
print(DHC_import.head())
print(DHC_import.info())

## Clean DHC data
All 322 features will need:  
to be renamed (for clarity) or  
to be dropped (for redundency)
Project will prioritize 'percent' variables, scale is improved over 'count'.

In [ ]:
# Create new working dataframe
DHC_clean = DHC_import.copy()

# Remove Puerto Rico, rows where GEOID is US72000 or greater
DHC_clean = DHC_clean[~DHC_clean['Geography'].str.startswith('0500000US72')]

# Rename Geography and Geographic Area Name
DHC_clean = DHC_clean.rename(columns={
    'Geography': 'GEOID',
    'Geographic Area Name': 'County'})
DHC_clean['GEOID'] = DHC_clean['GEOID'].str[-5:]

# Confirm
print(DHC_clean.info())

Continue to CLEAN age data  
[previously looked at 10 and 15 year groups (generation) during original analysis of Pennsylvania only data (before expanding to nationwide study)]

In [ ]:
# Rename age variables to keep, drop remaining
DHC_clean = DHC_clean.rename(columns={
    'Count!!SEX AND AGE!!Total population': 'Pop_total',
    'Count!!SEX AND AGE!!Total population!!Under 5 years': 'Total_U5',
    'Count!!SEX AND AGE!!Total population!!5 to 9 years': 'Total_5_9',
    'Count!!SEX AND AGE!!Total population!!10 to 14 years': 'Total_10_14',
    'Count!!SEX AND AGE!!Total population!!15 to 19 years': 'Total_15_19',
    'Count!!SEX AND AGE!!Male population': 'Male_total',
    'Count!!SEX AND AGE!!Male population!!Under 5 years': 'Male_U5',
    'Count!!SEX AND AGE!!Male population!!5 to 9 years': 'Male_5_9',
    'Count!!SEX AND AGE!!Male population!!10 to 14 years': 'Male_10_14',
    'Count!!SEX AND AGE!!Male population!!15 to 19 years': 'Male_15_19',
    'Count!!SEX AND AGE!!Female population': 'Female_total',
    'Count!!SEX AND AGE!!Female population!!Under 5 years': 'Female_U5',
    'Count!!SEX AND AGE!!Female population!!5 to 9 years': 'Female_5_9',
    'Count!!SEX AND AGE!!Female population!!10 to 14 years': 'Female_10_14',
    'Count!!SEX AND AGE!!Female population!!15 to 19 years': 'Female_15_19',
    'Count!!SEX AND AGE!!Total population!!Selected Age Categories!!18 years and over': 'Total_18+',
    'Count!!SEX AND AGE!!Male population!!Selected Age Categories!!18 years and over': 'Male_18+',
    'Count!!SEX AND AGE!!Female population!!Selected Age Categories!!18 years and over': 'Female_18+'})

columns_age_drop = [
    'Count!!SEX AND AGE!!Total population!!20 to 24 years',
    'Count!!SEX AND AGE!!Total population!!25 to 29 years',
    'Count!!SEX AND AGE!!Total population!!30 to 34 years',
    'Count!!SEX AND AGE!!Total population!!35 to 39 years',
    'Count!!SEX AND AGE!!Total population!!40 to 44 years',
    'Count!!SEX AND AGE!!Total population!!45 to 49 years',
    'Count!!SEX AND AGE!!Total population!!50 to 54 years',
    'Count!!SEX AND AGE!!Total population!!55 to 59 years',
    'Count!!SEX AND AGE!!Total population!!60 to 64 years',
    'Count!!SEX AND AGE!!Total population!!65 to 69 years',
    'Count!!SEX AND AGE!!Total population!!70 to 74 years',
    'Count!!SEX AND AGE!!Total population!!75 to 79 years',
    'Count!!SEX AND AGE!!Total population!!80 to 84 years',
    'Count!!SEX AND AGE!!Total population!!85 years and over',
    'Count!!SEX AND AGE!!Total population!!Selected Age Categories!!16 years and over',
    'Count!!SEX AND AGE!!Total population!!Selected Age Categories!!21 years and over',
    'Count!!SEX AND AGE!!Total population!!Selected Age Categories!!62 years and over',
    'Count!!SEX AND AGE!!Total population!!Selected Age Categories!!65 years and over',
    'Count!!SEX AND AGE!!Male population!!20 to 24 years',
    'Count!!SEX AND AGE!!Male population!!25 to 29 years',
    'Count!!SEX AND AGE!!Male population!!30 to 34 years',
    'Count!!SEX AND AGE!!Male population!!35 to 39 years',
    'Count!!SEX AND AGE!!Male population!!40 to 44 years',
    'Count!!SEX AND AGE!!Male population!!45 to 49 years',
    'Count!!SEX AND AGE!!Male population!!50 to 54 years',
    'Count!!SEX AND AGE!!Male population!!55 to 59 years',
    'Count!!SEX AND AGE!!Male population!!60 to 64 years',
    'Count!!SEX AND AGE!!Male population!!65 to 69 years',
    'Count!!SEX AND AGE!!Male population!!70 to 74 years',
    'Count!!SEX AND AGE!!Male population!!75 to 79 years',
    'Count!!SEX AND AGE!!Male population!!80 to 84 years',
    'Count!!SEX AND AGE!!Male population!!85 years and over',
    'Count!!SEX AND AGE!!Male population!!Selected Age Categories!!16 years and over',
    'Count!!SEX AND AGE!!Male population!!Selected Age Categories!!21 years and over',
    'Count!!SEX AND AGE!!Male population!!Selected Age Categories!!62 years and over',
    'Count!!SEX AND AGE!!Male population!!Selected Age Categories!!65 years and over',
    'Count!!SEX AND AGE!!Female population!!20 to 24 years',
    'Count!!SEX AND AGE!!Female population!!25 to 29 years',
    'Count!!SEX AND AGE!!Female population!!30 to 34 years',
    'Count!!SEX AND AGE!!Female population!!35 to 39 years',
    'Count!!SEX AND AGE!!Female population!!40 to 44 years',
    'Count!!SEX AND AGE!!Female population!!45 to 49 years',
    'Count!!SEX AND AGE!!Female population!!50 to 54 years',
    'Count!!SEX AND AGE!!Female population!!55 to 59 years',
    'Count!!SEX AND AGE!!Female population!!60 to 64 years',
    'Count!!SEX AND AGE!!Female population!!65 to 69 years',
    'Count!!SEX AND AGE!!Female population!!70 to 74 years',
    'Count!!SEX AND AGE!!Female population!!75 to 79 years',
    'Count!!SEX AND AGE!!Female population!!80 to 84 years',
    'Count!!SEX AND AGE!!Female population!!85 years and over',
    'Count!!SEX AND AGE!!Female population!!Selected Age Categories!!16 years and over',
    'Count!!SEX AND AGE!!Female population!!Selected Age Categories!!21 years and over',
    'Count!!SEX AND AGE!!Female population!!Selected Age Categories!!62 years and over',
    'Count!!SEX AND AGE!!Female population!!Selected Age Categories!!65 years and over']

# Drop the specified columns
DHC_clean.drop(columns=columns_age_drop, inplace=True)

In [ ]:
# Rename percent age groups
DHC_clean = DHC_clean.rename(columns={
    'Percent!!SEX AND AGE!!Total population!!20 to 24 years': '%TOTAL_20_24',
    'Percent!!SEX AND AGE!!Total population!!25 to 29 years': '%TOTAL_25_29',
    'Percent!!SEX AND AGE!!Total population!!30 to 34 years': '%TOTAL_30_34',
    'Percent!!SEX AND AGE!!Total population!!35 to 39 years': '%TOTAL_35_39',
    'Percent!!SEX AND AGE!!Total population!!40 to 44 years': '%TOTAL_40_44',
    'Percent!!SEX AND AGE!!Total population!!45 to 49 years': '%TOTAL_45_49',
    'Percent!!SEX AND AGE!!Total population!!50 to 54 years': '%TOTAL_50_54',
    'Percent!!SEX AND AGE!!Total population!!55 to 59 years': '%TOTAL_55_59',
    'Percent!!SEX AND AGE!!Total population!!60 to 64 years': '%TOTAL_60_64',
    'Percent!!SEX AND AGE!!Total population!!65 to 69 years': '%TOTAL_65_69',
    'Percent!!SEX AND AGE!!Total population!!70 to 74 years': '%TOTAL_70_74',
    'Percent!!SEX AND AGE!!Total population!!75 to 79 years': '%TOTAL_75_79',
    'Percent!!SEX AND AGE!!Total population!!80 to 84 years': '%TOTAL_80_84',
    'Percent!!SEX AND AGE!!Total population!!85 years and over': '%TOTAL_85+',
    'Percent!!SEX AND AGE!!Male population!!20 to 24 years': '%MALE_20_24',
    'Percent!!SEX AND AGE!!Male population!!25 to 29 years': '%MALE_25_29',
    'Percent!!SEX AND AGE!!Male population!!30 to 34 years': '%MALE_30_34',
    'Percent!!SEX AND AGE!!Male population!!35 to 39 years': '%MALE_35_39',
    'Percent!!SEX AND AGE!!Male population!!40 to 44 years': '%MALE_40_44',
    'Percent!!SEX AND AGE!!Male population!!45 to 49 years': '%MALE_45_49',
    'Percent!!SEX AND AGE!!Male population!!50 to 54 years': '%MALE_50_54',
    'Percent!!SEX AND AGE!!Male population!!55 to 59 years': '%MALE_55_59',
    'Percent!!SEX AND AGE!!Male population!!60 to 64 years': '%MALE_60_64',
    'Percent!!SEX AND AGE!!Male population!!65 to 69 years': '%MALE_65_69',
    'Percent!!SEX AND AGE!!Male population!!70 to 74 years': '%MALE_70_74',
    'Percent!!SEX AND AGE!!Male population!!75 to 79 years': '%MALE_75_79',
    'Percent!!SEX AND AGE!!Male population!!80 to 84 years': '%MALE_80_84',
    'Percent!!SEX AND AGE!!Male population!!85 years and over': '%MALE_85+',
    'Percent!!SEX AND AGE!!Female population!!20 to 24 years': '%FEMALE_20_24',
    'Percent!!SEX AND AGE!!Female population!!25 to 29 years': '%FEMALE_25_29',
    'Percent!!SEX AND AGE!!Female population!!30 to 34 years': '%FEMALE_30_34',
    'Percent!!SEX AND AGE!!Female population!!35 to 39 years': '%FEMALE_35_39',
    'Percent!!SEX AND AGE!!Female population!!40 to 44 years': '%FEMALE_40_44',
    'Percent!!SEX AND AGE!!Female population!!45 to 49 years': '%FEMALE_45_49',
    'Percent!!SEX AND AGE!!Female population!!50 to 54 years': '%FEMALE_50_54',
    'Percent!!SEX AND AGE!!Female population!!55 to 59 years': '%FEMALE_55_59',
    'Percent!!SEX AND AGE!!Female population!!60 to 64 years': '%FEMALE_60_64',
    'Percent!!SEX AND AGE!!Female population!!65 to 69 years': '%FEMALE_65_69',
    'Percent!!SEX AND AGE!!Female population!!70 to 74 years': '%FEMALE_70_74',
    'Percent!!SEX AND AGE!!Female population!!75 to 79 years': '%FEMALE_75_79',
    'Percent!!SEX AND AGE!!Female population!!80 to 84 years': '%FEMALE_80_84',
    'Percent!!SEX AND AGE!!Female population!!85 years and over': '%FEMALE_85+'})

Continue to CLEAN median age data

In [ ]:
# Rename median variables
DHC_clean = DHC_clean.rename(columns={
    'Count!!MEDIAN AGE BY SEX!!Both sexes': 'MED_AGE',
    'Count!!MEDIAN AGE BY SEX!!Male': 'MED_AGE_M',
    'Count!!MEDIAN AGE BY SEX!!Female': 'MED_AGE_F'})

# Reorder columns to move 'Median Age' next to population totals
cols = DHC_clean.columns.tolist()
cols.insert(20, cols.pop(cols.index('MED_AGE')))
cols.insert(21, cols.pop(cols.index('MED_AGE_M')))
cols.insert(22, cols.pop(cols.index('MED_AGE_F')))
DHC_clean = DHC_clean[cols]

Continue to CLEAN race data

In [ ]:
# Rename race variables to keep, drop remaining
DHC_clean = DHC_clean.rename(columns={
    'Percent!!HISPANIC OR LATINO BY RACE!!Total population!!Not Hispanic or Latino!!White alone': '%RACE_White',
    'Percent!!HISPANIC OR LATINO BY RACE!!Total population!!Not Hispanic or Latino!!Black or African American alone': '%RACE_Black',
    'Percent!!HISPANIC OR LATINO BY RACE!!Total population!!Hispanic or Latino': '%RACE_Latino',
    'Percent!!HISPANIC OR LATINO BY RACE!!Total population!!Not Hispanic or Latino!!American Indian and Alaska Native alone': '%RACE_Native',
    'Percent!!HISPANIC OR LATINO BY RACE!!Total population!!Not Hispanic or Latino!!Asian alone': '%RACE_Asian',
    'Percent!!HISPANIC OR LATINO BY RACE!!Total population!!Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone': '%RACE_HI_PI',
    'Percent!!HISPANIC OR LATINO BY RACE!!Total population!!Not Hispanic or Latino!!Some Other Race alone': '%RACE_Other',
    'Percent!!HISPANIC OR LATINO BY RACE!!Total population!!Not Hispanic or Latino!!Two or More Races': '%RACE_Mixed'})

columns_race_drop = [
    'Count!!RACE!!Total population',
    'Count!!RACE!!Total population!!One Race',
    'Count!!RACE!!Total population!!One Race!!White',
    'Count!!RACE!!Total population!!One Race!!Black or African American',
    'Count!!RACE!!Total population!!One Race!!American Indian and Alaska Native',
    'Count!!RACE!!Total population!!One Race!!Asian',
    'Count!!RACE!!Total population!!One Race!!Native Hawaiian and Other Pacific Islander',
    'Count!!RACE!!Total population!!One Race!!Some Other Race',
    'Count!!RACE!!Total population!!Two or More Races',
    'Count!!TOTAL RACES TALLIED [1]!!Total races tallied',
    'Count!!TOTAL RACES TALLIED [1]!!Total races tallied!!White alone or in combination with one or more other races',
    'Count!!TOTAL RACES TALLIED [1]!!Total races tallied!!Black or African American alone or in combination with one or more other races',
    'Count!!TOTAL RACES TALLIED [1]!!Total races tallied!!American Indian and Alaska Native alone or in combination with one or more other races',
    'Count!!TOTAL RACES TALLIED [1]!!Total races tallied!!Asian alone or in combination with one or more other races',
    'Count!!TOTAL RACES TALLIED [1]!!Total races tallied!!Native Hawaiian and Other Pacific Islander alone or in combination with one or more other races',
    'Count!!TOTAL RACES TALLIED [1]!!Total races tallied!!Some Other Race alone or in combination with one or more other races',
    'Count!!HISPANIC OR LATINO!!Total population',
    'Count!!HISPANIC OR LATINO!!Total population!!Hispanic or Latino (of any race)',
    'Count!!HISPANIC OR LATINO!!Total population!!Not Hispanic or Latino',
    'Count!!HISPANIC OR LATINO BY RACE!!Total population','Count!!HISPANIC OR LATINO BY RACE!!Total population!!Hispanic or Latino',
    'Count!!HISPANIC OR LATINO BY RACE!!Total population!!Hispanic or Latino!!White alone',
    'Count!!HISPANIC OR LATINO BY RACE!!Total population!!Hispanic or Latino!!Black or African American alone',
    'Count!!HISPANIC OR LATINO BY RACE!!Total population!!Hispanic or Latino!!American Indian and Alaska Native alone',
    'Count!!HISPANIC OR LATINO BY RACE!!Total population!!Hispanic or Latino!!Asian alone',
    'Count!!HISPANIC OR LATINO BY RACE!!Total population!!Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone',
    'Count!!HISPANIC OR LATINO BY RACE!!Total population!!Hispanic or Latino!!Some Other Race alone',
    'Count!!HISPANIC OR LATINO BY RACE!!Total population!!Hispanic or Latino!!Two or More Races',
    'Count!!HISPANIC OR LATINO BY RACE!!Total population!!Not Hispanic or Latino',
    'Count!!HISPANIC OR LATINO BY RACE!!Total population!!Not Hispanic or Latino!!White alone',
    'Count!!HISPANIC OR LATINO BY RACE!!Total population!!Not Hispanic or Latino!!Black or African American alone',
    'Count!!HISPANIC OR LATINO BY RACE!!Total population!!Not Hispanic or Latino!!American Indian and Alaska Native alone',
    'Count!!HISPANIC OR LATINO BY RACE!!Total population!!Not Hispanic or Latino!!Asian alone',
    'Count!!HISPANIC OR LATINO BY RACE!!Total population!!Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone',
    'Count!!HISPANIC OR LATINO BY RACE!!Total population!!Not Hispanic or Latino!!Some Other Race alone',
    'Count!!HISPANIC OR LATINO BY RACE!!Total population!!Not Hispanic or Latino!!Two or More Races']

# Drop the specified columns
DHC_clean.drop(columns=columns_race_drop, inplace=True)

Continue to CLEAN relationship data

In [ ]:
# Rename relationship variables to keep, drop remaining
DHC_clean = DHC_clean.rename(columns={
    'Percent!!RELATIONSHIP!!Total population!!In households!!Opposite-sex spouse': '%REL_OP_SEX_MAR',
    'Percent!!RELATIONSHIP!!Total population!!In households!!Same-sex spouse': '%REL_S_SEX_MAR',
    'Percent!!RELATIONSHIP!!Total population!!In households!!Opposite-sex unmarried partner': '%REL_OP_SEX_UNMAR',
    'Percent!!RELATIONSHIP!!Total population!!In households!!Same-sex unmarried partner': '%REL_S_SEX_UNMAR',
    'Percent!!RELATIONSHIP!!Total population!!In households!!Other relatives': '%REL_W_RELATIVES',
    'Percent!!RELATIONSHIP!!Total population!!In households!!Nonrelatives': '%REL_NON_REL',
    'Percent!!RELATIONSHIP!!Total population!!In group quarters!!Institutionalized population:!!Male': '%REL_MALE_JAILED',
    'Percent!!RELATIONSHIP!!Total population!!In group quarters!!Institutionalized population:!!Female': '%REL_FEMALE_JAILED',
    'Percent!!RELATIONSHIP!!Total population!!In group quarters!!Noninstitutionalized population:!!Male': '%REL_MALE_GRP_DORM',
    'Percent!!RELATIONSHIP!!Total population!!In group quarters!!Noninstitutionalized population:!!Female': '%REL_FEMALE_GRP_DORM'})

columns_rel_drop = [
    'Count!!RELATIONSHIP!!Total population',
    'Count!!RELATIONSHIP!!Total population!!In households',
    'Count!!RELATIONSHIP!!Total population!!In households!!Householder',
    'Count!!RELATIONSHIP!!Total population!!In households!!Opposite-sex spouse',
    'Count!!RELATIONSHIP!!Total population!!In households!!Same-sex spouse',
    'Count!!RELATIONSHIP!!Total population!!In households!!Opposite-sex unmarried partner',
    'Count!!RELATIONSHIP!!Total population!!In households!!Same-sex unmarried partner',
    'Count!!RELATIONSHIP!!Total population!!In households!!Child [2]',
    'Count!!RELATIONSHIP!!Total population!!In households!!Child [2]!!Under 18 years',
    'Count!!RELATIONSHIP!!Total population!!In households!!Grandchild',
    'Count!!RELATIONSHIP!!Total population!!In households!!Grandchild!!Under 18 years',
    'Count!!RELATIONSHIP!!Total population!!In households!!Other relatives',
    'Count!!RELATIONSHIP!!Total population!!In households!!Nonrelatives',
    'Count!!RELATIONSHIP!!Total population!!In group quarters',
    'Count!!RELATIONSHIP!!Total population!!In group quarters!!Institutionalized population:',
    'Count!!RELATIONSHIP!!Total population!!In group quarters!!Institutionalized population:!!Male',
    'Count!!RELATIONSHIP!!Total population!!In group quarters!!Institutionalized population:!!Female',
    'Count!!RELATIONSHIP!!Total population!!In group quarters!!Noninstitutionalized population:',
    'Count!!RELATIONSHIP!!Total population!!In group quarters!!Noninstitutionalized population:!!Male',
    'Count!!RELATIONSHIP!!Total population!!In group quarters!!Noninstitutionalized population:!!Female']

# Drop the specified columns
DHC_clean.drop(columns=columns_rel_drop, inplace=True)

Continue to CLEAN household data

In [ ]:
# Rename household variables to keep, drop remaining
DHC_clean = DHC_clean.rename(columns={
    'Percent!!HOUSEHOLDS BY TYPE!!Total households!!Married couple household': '%HH_MARRIED',
    'Percent!!HOUSEHOLDS BY TYPE!!Total households!!Married couple household!!With own children under 18 [3]': '%HH_MAR_W_KIDS',
    'Percent!!HOUSEHOLDS BY TYPE!!Total households!!Cohabiting couple household': '%HH_NOT_MAR',
    'Percent!!HOUSEHOLDS BY TYPE!!Total households!!Cohabiting couple household!!With own children under 18 [3]': '%HH_NOT_MAR_W_KIDS',
    'Percent!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse or partner present:!!Living alone': '%HH_MALE_ALONE',
    'Percent!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse or partner present:!!Living alone!!65 years and over': '%HH_MALE_65+',
    'Percent!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse or partner present:!!With own children under 18 [3]': '%HH_MALE_W_KIDS',
    'Percent!!HOUSEHOLDS BY TYPE!!Total households!!Female householder, no spouse or partner present:!!Living alone': '%HH_FEMALE_ALONE',
    'Percent!!HOUSEHOLDS BY TYPE!!Total households!!Female householder, no spouse or partner present:!!Living alone!!65 years and over': '%HH_FEMALE_65+',
    'Percent!!HOUSEHOLDS BY TYPE!!Total households!!Female householder, no spouse or partner present:!!With own children under 18 [3]': '%HH_FEMALE_W_KIDS'})

columns_hhold_drop = [
    'Count!!HOUSEHOLDS BY TYPE!!Total households',
    'Count!!HOUSEHOLDS BY TYPE!!Total households!!Married couple household',
    'Count!!HOUSEHOLDS BY TYPE!!Total households!!Married couple household!!With own children under 18 [3]',
    'Count!!HOUSEHOLDS BY TYPE!!Total households!!Cohabiting couple household',
    'Count!!HOUSEHOLDS BY TYPE!!Total households!!Cohabiting couple household!!With own children under 18 [3]',
    'Count!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse or partner present:',
    'Count!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse or partner present:!!Living alone',
    'Count!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse or partner present:!!Living alone!!65 years and over',
    'Count!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse or partner present:!!With own children under 18 [3]',
    'Count!!HOUSEHOLDS BY TYPE!!Total households!!Female householder, no spouse or partner present:',
    'Count!!HOUSEHOLDS BY TYPE!!Total households!!Female householder, no spouse or partner present:!!Living alone',
    'Count!!HOUSEHOLDS BY TYPE!!Total households!!Female householder, no spouse or partner present:!!With own children under 18 [3]',
    'Count!!HOUSEHOLDS BY TYPE!!Total households!!Female householder, no spouse or partner present:!!Living alone!!65 years and over',
    'Count!!HOUSEHOLDS BY TYPE!!Total households!!Households with individuals under 18 years',
    'Count!!HOUSEHOLDS BY TYPE!!Total households!!Households with individuals 65 years and over']

# Drop the specified columns
DHC_clean.drop(columns=columns_hhold_drop, inplace=True)

Continue to CLEAN housing data

In [ ]:
# Rename housing variables to keep, drop remaining
DHC_clean = DHC_clean.rename(columns={
    'Percent!!HOUSING TENURE!!Occupied housing units!!Owner-occupied housing units': '%OWN_HOME',
    'Percent!!HOUSING TENURE!!Occupied housing units!!Renter-occupied housing units': '%RENT_HOME'})

columns_housing_drop = [
    'Count!!HOUSING OCCUPANCY!!Total housing units',
    'Count!!HOUSING OCCUPANCY!!Total housing units!!Occupied housing units',
    'Count!!HOUSING OCCUPANCY!!Total housing units!!Vacant housing units',
    'Count!!HOUSING OCCUPANCY!!Total housing units!!Vacant housing units!!For rent',
    'Count!!HOUSING OCCUPANCY!!Total housing units!!Vacant housing units!!Rented, not occupied',
    'Count!!HOUSING OCCUPANCY!!Total housing units!!Vacant housing units!!For sale only',
    'Count!!HOUSING OCCUPANCY!!Total housing units!!Vacant housing units!!Sold, not occupied',
    'Count!!HOUSING OCCUPANCY!!Total housing units!!Vacant housing units!!For seasonal, recreational, or occasional use',
    'Count!!HOUSING OCCUPANCY!!Total housing units!!Vacant housing units!!All other vacants',
    'Count!!VACANCY RATES!!Homeowner vacancy rate (percent) [4]',
    'Count!!VACANCY RATES!!Rental vacancy rate (percent) [5]',
    'Count!!HOUSING TENURE!!Occupied housing units',
    'Count!!HOUSING TENURE!!Occupied housing units!!Owner-occupied housing units',
    'Count!!HOUSING TENURE!!Occupied housing units!!Renter-occupied housing units']

# Drop the specified columns
DHC_clean.drop(columns=columns_housing_drop, inplace=True)

Continue to CLEAN percentage data

In [ ]:
# Drop remaining percentage variables
columns_percent_drop = [
    'Percent!!SEX AND AGE!!Total population',
    'Percent!!SEX AND AGE!!Total population!!Under 5 years',
    'Percent!!SEX AND AGE!!Total population!!5 to 9 years',
    'Percent!!SEX AND AGE!!Total population!!10 to 14 years',
    'Percent!!SEX AND AGE!!Total population!!15 to 19 years',
    'Percent!!SEX AND AGE!!Total population!!Selected Age Categories!!16 years and over',
    'Percent!!SEX AND AGE!!Total population!!Selected Age Categories!!18 years and over',
    'Percent!!SEX AND AGE!!Total population!!Selected Age Categories!!21 years and over',
    'Percent!!SEX AND AGE!!Total population!!Selected Age Categories!!62 years and over',
    'Percent!!SEX AND AGE!!Total population!!Selected Age Categories!!65 years and over',
    'Percent!!SEX AND AGE!!Male population',
    'Percent!!SEX AND AGE!!Male population!!Under 5 years',
    'Percent!!SEX AND AGE!!Male population!!5 to 9 years',
    'Percent!!SEX AND AGE!!Male population!!10 to 14 years',
    'Percent!!SEX AND AGE!!Male population!!15 to 19 years',
    'Percent!!SEX AND AGE!!Male population!!Selected Age Categories!!16 years and over',
    'Percent!!SEX AND AGE!!Male population!!Selected Age Categories!!18 years and over',
    'Percent!!SEX AND AGE!!Male population!!Selected Age Categories!!21 years and over',
    'Percent!!SEX AND AGE!!Male population!!Selected Age Categories!!62 years and over',
    'Percent!!SEX AND AGE!!Male population!!Selected Age Categories!!65 years and over',
    'Percent!!SEX AND AGE!!Female population',
    'Percent!!SEX AND AGE!!Female population!!Under 5 years',
    'Percent!!SEX AND AGE!!Female population!!5 to 9 years',
    'Percent!!SEX AND AGE!!Female population!!10 to 14 years',
    'Percent!!SEX AND AGE!!Female population!!15 to 19 years',
    'Percent!!SEX AND AGE!!Female population!!Selected Age Categories!!16 years and over',
    'Percent!!SEX AND AGE!!Female population!!Selected Age Categories!!18 years and over',
    'Percent!!SEX AND AGE!!Female population!!Selected Age Categories!!21 years and over',
    'Percent!!SEX AND AGE!!Female population!!Selected Age Categories!!62 years and over',
    'Percent!!SEX AND AGE!!Female population!!Selected Age Categories!!65 years and over',
    'Percent!!MEDIAN AGE BY SEX!!Both sexes',
    'Percent!!MEDIAN AGE BY SEX!!Male',
    'Percent!!MEDIAN AGE BY SEX!!Female',
    'Percent!!RACE!!Total population',
    'Percent!!RACE!!Total population!!One Race',
    'Percent!!RACE!!Total population!!One Race!!White',
    'Percent!!RACE!!Total population!!One Race!!Black or African American',
    'Percent!!RACE!!Total population!!One Race!!American Indian and Alaska Native',
    'Percent!!RACE!!Total population!!One Race!!Asian',
    'Percent!!RACE!!Total population!!One Race!!Native Hawaiian and Other Pacific Islander',
    'Percent!!RACE!!Total population!!One Race!!Some Other Race',
    'Percent!!RACE!!Total population!!Two or More Races',
    'Percent!!TOTAL RACES TALLIED [1]!!Total races tallied',
    'Percent!!TOTAL RACES TALLIED [1]!!Total races tallied!!White alone or in combination with one or more other races',
    'Percent!!TOTAL RACES TALLIED [1]!!Total races tallied!!Black or African American alone or in combination with one or more other races',
    'Percent!!TOTAL RACES TALLIED [1]!!Total races tallied!!American Indian and Alaska Native alone or in combination with one or more other races',
    'Percent!!TOTAL RACES TALLIED [1]!!Total races tallied!!Asian alone or in combination with one or more other races',
    'Percent!!TOTAL RACES TALLIED [1]!!Total races tallied!!Native Hawaiian and Other Pacific Islander alone or in combination with one or more other races',
    'Percent!!TOTAL RACES TALLIED [1]!!Total races tallied!!Some Other Race alone or in combination with one or more other races',
    'Percent!!HISPANIC OR LATINO!!Total population',
    'Percent!!HISPANIC OR LATINO!!Total population!!Hispanic or Latino (of any race)',
    'Percent!!HISPANIC OR LATINO!!Total population!!Not Hispanic or Latino',
    'Percent!!HISPANIC OR LATINO BY RACE!!Total population',
    'Percent!!HISPANIC OR LATINO BY RACE!!Total population!!Hispanic or Latino!!White alone',
    'Percent!!HISPANIC OR LATINO BY RACE!!Total population!!Hispanic or Latino!!Black or African American alone',
    'Percent!!HISPANIC OR LATINO BY RACE!!Total population!!Hispanic or Latino!!American Indian and Alaska Native alone',
    'Percent!!HISPANIC OR LATINO BY RACE!!Total population!!Hispanic or Latino!!Asian alone',
    'Percent!!HISPANIC OR LATINO BY RACE!!Total population!!Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone',
    'Percent!!HISPANIC OR LATINO BY RACE!!Total population!!Hispanic or Latino!!Some Other Race alone',
    'Percent!!HISPANIC OR LATINO BY RACE!!Total population!!Hispanic or Latino!!Two or More Races',
    'Percent!!HISPANIC OR LATINO BY RACE!!Total population!!Not Hispanic or Latino',
    'Percent!!RELATIONSHIP!!Total population',
    'Percent!!RELATIONSHIP!!Total population!!In households',
    'Percent!!RELATIONSHIP!!Total population!!In households!!Householder',
    'Percent!!RELATIONSHIP!!Total population!!In households!!Child [2]',
    'Percent!!RELATIONSHIP!!Total population!!In households!!Child [2]!!Under 18 years',
    'Percent!!RELATIONSHIP!!Total population!!In households!!Grandchild',
    'Percent!!RELATIONSHIP!!Total population!!In households!!Grandchild!!Under 18 years',
    'Percent!!RELATIONSHIP!!Total population!!In group quarters',
    'Percent!!RELATIONSHIP!!Total population!!In group quarters!!Institutionalized population:',
    'Percent!!RELATIONSHIP!!Total population!!In group quarters!!Noninstitutionalized population:',
    'Percent!!HOUSEHOLDS BY TYPE!!Total households',
    'Percent!!HOUSEHOLDS BY TYPE!!Total households!!Male householder, no spouse or partner present:',
    'Percent!!HOUSEHOLDS BY TYPE!!Total households!!Female householder, no spouse or partner present:',
    'Percent!!HOUSEHOLDS BY TYPE!!Total households!!Households with individuals under 18 years',
    'Percent!!HOUSEHOLDS BY TYPE!!Total households!!Households with individuals 65 years and over',
    'Percent!!HOUSING OCCUPANCY!!Total housing units',
    'Percent!!HOUSING OCCUPANCY!!Total housing units!!Occupied housing units',
    'Percent!!HOUSING OCCUPANCY!!Total housing units!!Vacant housing units',
    'Percent!!HOUSING OCCUPANCY!!Total housing units!!Vacant housing units!!For rent',
    'Percent!!HOUSING OCCUPANCY!!Total housing units!!Vacant housing units!!Rented, not occupied',
    'Percent!!HOUSING OCCUPANCY!!Total housing units!!Vacant housing units!!For sale only',
    'Percent!!HOUSING OCCUPANCY!!Total housing units!!Vacant housing units!!Sold, not occupied',
    'Percent!!HOUSING OCCUPANCY!!Total housing units!!Vacant housing units!!For seasonal, recreational, or occasional use',
    'Percent!!HOUSING OCCUPANCY!!Total housing units!!Vacant housing units!!All other vacants',
    'Percent!!VACANCY RATES!!Homeowner vacancy rate (percent) [4]',
    'Percent!!VACANCY RATES!!Rental vacancy rate (percent) [5]',
    'Percent!!HOUSING TENURE!!Occupied housing units']

# Drop the specified columns
DHC_clean.drop(columns=columns_percent_drop, inplace=True)

#Confirm
#print(DHC_clean.info())

## Transform DHC data

Create new 'Under 18' and '18-19' age groups

In [ ]:
# Create new working dataframe
DHC_transform = DHC_clean.copy()

# Calculate 'Under 18' by subtracting '18 years and over' from 'totals'
DHC_transform['Total_U18'] = DHC_transform[
    'Pop_total'] - DHC_transform['Total_18+']
DHC_transform['Male_U18'] = DHC_transform[
    'Male_total'] - DHC_transform['Male_18+']
DHC_transform['Female_U18'] = DHC_transform[
    'Female_total'] - DHC_transform['Female_18+']

# Calculate 'Total_18-19' by adding all ages 0-19 and subtracting U18
DHC_transform['Total_18_19'] = (DHC_transform['Total_U5'] +
    DHC_transform['Total_5_9'] + DHC_transform['Total_10_14'] +
    DHC_transform['Total_15_19'] - DHC_transform['Total_U18'])

# Repeat for Male 18-19
DHC_transform['Male_18_19'] = (DHC_transform['Male_U5'] +
    DHC_transform['Male_5_9'] + DHC_transform['Male_10_14'] +
    DHC_transform['Male_15_19'] - DHC_transform['Male_U18'])

# Repeat for Female 18-19
DHC_transform['Female_18_19'] = (DHC_transform['Female_U5'] +
    DHC_transform['Female_5_9'] + DHC_transform['Female_10_14'] +
    DHC_transform['Female_15_19'] - DHC_transform['Female_U18'])

# Calculate '%_18-19' by dividing by 'totals'
DHC_transform['%TOTAL_18_19'] = (
    DHC_transform['Total_18_19'] / DHC_transform['Pop_total']* 100).round(2)
DHC_transform['%MALE_18_19'] = (
    DHC_transform['Male_18_19'] / DHC_transform['Male_total']* 100).round(2)
DHC_transform['%FEMALE_18_19'] = (
    DHC_transform['Female_18_19'] / DHC_transform['Female_total']* 100).round(2)

# Can now drop these columns
columns_tform_drop = [
    'Total_U5', 'Male_U5', 'Female_U5',
    'Total_5_9', 'Male_5_9', 'Female_5_9',
    'Total_10_14', 'Male_10_14', 'Female_10_14',
    'Total_15_19', 'Male_15_19', 'Female_15_19',
    'Total_18+', 'Male_18+', 'Female_18+',
    'Total_U18', 'Male_U18', 'Female_U18',
    'Total_18_19', 'Male_18_19', 'Female_18_19']
DHC_transform.drop(columns=columns_tform_drop, inplace=True)

# Reorder columns to move '18-19' before '20-24'
cols = DHC_transform.columns.tolist()
cols.insert(8, cols.pop(cols.index('%TOTAL_18_19')))
cols.insert(23, cols.pop(cols.index('%MALE_18_19')))
cols.insert(38, cols.pop(cols.index('%FEMALE_18_19')))
DHC_transform = DHC_transform[cols]

#Confirm
#pd.set_option('display.max_columns', None)
#print(DHC_transform.head())
#print(DHC_transform.info())

## Save DHC cleaned data

In [ ]:
# Create the tidy dataframe
DHC_tidy = DHC_transform.copy()

DHC_tidy.to_csv('DHC_tidy.csv', index=False)

# Import P2 data (2 of 3)

P2 data is the population living in urban or rural (PUR) areas within each county **[number of households also available (H2)]**.  
For the 2020 Census, an urban area will comprise a densely settled core of census blocks that meet minimum population density requirements. This includes adjacent territory containing non-residential urban land uses. To qualify as an urban area, the territory identified according to criteria must have a population of at least 5,000. *(Note on Alaska: See accompanying 'Alaska County' amalgamation file on github for method used to match census area to state senate district. PUR datafile combines 30 census areas into 14 'County_fips' created for this analysis)*. Also dropped Kalawao County, Hawaii: 82 rural residents, none of them voted, dropping will align it with MEDSL file when Kalawao county_fips (15005) is cleaned from MEDSL data.    
https://data.census.gov/all?q=urban+and+rural&g=010XX00US$0500000

In [ ]:
# Import next dataset
PUR_import = pd.read_csv(
    'DECENNIALDHC2020.P2-AKfix.csv', header=1)

# Inspect
print(PUR_import.info())
print(PUR_import.head())

## Clean PUR data

In [ ]:
# Create new working dataframe
PUR_clean = PUR_import.copy()

# Remove Puerto Rico: rows where GEOID is US72000 or greater
PUR_clean = PUR_clean[~PUR_clean['Geography'].str.startswith('0500000US72')]

# Rename variables to keep and drop remaining
PUR_clean = PUR_clean.rename(columns={
    'Geography': 'GEOID',
    ' !!Total:': 'Pop_total',
    ' !!Total:!!Urban': 'Pop_Urban',
    ' !!Total:!!Rural': 'Pop_Rural'})

PUR_clean['GEOID'] = PUR_clean['GEOID'].str[-5:]

# Calculate Urban percent
PUR_clean['%Urban_pop'] = (
    (PUR_clean['Pop_Urban'] / PUR_clean['Pop_total']) * 100).round(2)

# Drop the specified columns
columns_PUR_drop = ['Pop_total',
                    'Geographic Area Name',
                    ' !!Total:!!Not defined for this file']
PUR_clean.drop(columns=columns_PUR_drop, inplace=True)

# Confirm
#print(PUR_clean.info())

## Save PUR data

In [ ]:
# Create the tidy dataframe
PUR_tidy = PUR_clean.copy()

PUR_tidy.to_csv('PUR_tidy.csv', index=False)

# Import MEDSL data (3 of 3)
2020 general election results for most* (46) of the 50 states and D.C. downloaded from MEDSL (the MIT Election Data and Science Lab) https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/NT66Z3  

* ALASKA: voting data is not gathered by county, MEDSL 'county_fips' is empty. Used https://www.elections.alaska.gov/results/20GENR/Map/ Votes aggregated to state senate districts (1 - 40). See accompanying 'Alaska County' amalgamation file on github for method used to match 30 census areas to 40 state senate districts. MEDSL datafile uses 14 County_fips created for this analysis. Datafile only has the 4 variables that will be utilized here.

* INDIANA: MEDSL missing multiple county results. Used https://indianavoters.in.gov/ENRHistorical/ElectionResults  Datafile only has the 4 variables that will be utilized here, aggregated to the county level  

* NEW MEXICO: To protect the privacy of voters, New Mexico 'masks' vote totals in precinct results for candidates with small vote tallies. Used https://electionstats.sos.nm.gov/contest/13250  Datafile only has the 4 variables that will be utilized here, aggregated to the county level  

* NEVADA: To protect the privacy of voters, Nevada 'masks' vote totals in precinct results for candidates with 1-10 vote tallies. Used https://www.nvsos.gov/SOSelectionPages/results/2020StateWideGeneral/ElectionSummary.aspx  Datafile only has the 4 variables that will be utilized here, aggregated to the county level

##Pre-import processing Notes:  
The below adjustments were made to the MEDSL datafiles to standardize cleaning and processing.   

1. HAWAII: Adjusted DHC and PUR data regarding Kalawao County, Hawaii. Both have fips 15005, but there are no official votes cast, removed so all files align  

1. MAINE: Uniformed and Overseas Citizens Absentee Voting tallied seperately in 23000 fips, 23000 deleted to match DHC and PUR with votes added to 23005 (most populous county)  

1. MICHIGAN: MEDSL precinct data contains precinct '9999', which are 'statistical adjustments' rows. There were minor corrections needed to match official results at https://www.michigan.gov/sos/elections/election-results-and-data/candidate-listings-and-election-results-by-county  

1. MINNESOTA: 'DEMOCRATIC FARMER LABOR' party changed to 'DEMOCRAT'  

1. MISSOURI: MEDSL tallied Kansas City votes seperately in 36000 fips. Utillized https://www.sos.mo.gov/CMSImages/ElectionResultsStatistics/November3_2020GeneralElection.pdf to aportion some votes to Jackson County with remainder assigned to Clay County (official results not available on https://www.voteclaycountymo.gov/election-results), but totals match State official numbers  

1.  NEW YORK: 'CONSERVATIVE' party changed to 'REPUBLICAN'  
'WORKING FAMILIES' party changed to 'DEMOCRAT'  

1.  NORTH DAKOTA: 'DEMOCRATIC-NPL' party changed to 'DEMOCRAT' and 'county_fips' for OGLALA LAKOTA County changed from 46113 to 46102 to match data from DHC and PUR  

1.  OREGON: Sherman County included cadidate 'BALLOTS CAST' which totaled all votes in each precinct: Deleted  

1.  PENNSYLVANIA: 1 blank 'party_detailed' vote cast for Trump, party changed to 'REPUBLICAN'  

1.  VERMONT: 3 blank 'party_detailed' votes cast for Trump, party changed to 'REPUBLICAN'  
6 blank 'party_detailed' votes cast for Biden, party changed to 'DEMOCRAT'  

##Post-import cleaning Notes:
1.  All blanks in 'party_detailed' have been verified as writein votes cast for 'THIRD' party candidates  

2.  In Nov 2020, there were over 50 recognized political parties in the US.  
DEM and REP ballots accounted for 96% of total votes. Third parties accounted for 1-4% of the vote in each state. 'THIRD' will combine any vote NOT for Presidents Biden or Trump.  



In [ ]:
# Define list of all 51 voter CSV files to process (50 states plus D.C.)
file_list = glb.glob('2020-*-precinct-general.csv')

# Define function to read, select features, and clean a single CSV file
def process_file(file_path):

    try:
# Specify data types, let 'votes' be float during import
        dtype_spec = {'office': str, 'county_fips': str,
                      'party_detailed': str, 'votes': float}
        df = pd.read_csv(file_path, dtype=dtype_spec, low_memory=False)

# Filter for President in 'office' to avoid counting multiple votes per person
# Only analyze US Presidential race (it has the most voter participation)
        df = df[df['office'] == 'US PRESIDENT'].copy()
        df = df[['office', 'county_fips', 'party_detailed', 'votes']]
        df = df.rename(columns={
            'county_fips': 'GEOID',
            'party_detailed': 'PARTY',
            'votes': 'VOTES'})
        return df

    except Exception as e:
        print(f'Error processing {file_path}: {e}')
        return None

In [ ]:
# Iterate through the file list, apply function, and store dataframes
all_processed_dataframes = [
    process_file(file_path) for file_path in file_list]

# Filter out any None values if errors occurred during processing
all_processed_dataframes = [
    df for df in all_processed_dataframes if df is not None]

# Concatenate all processed files into single dataframe
US_combined = pd.concat(all_processed_dataframes, ignore_index=True)

# Confirm
#print(US_combined.info())
#print(US_combined.head())

## Clean MEDSL data

In [ ]:
# Fill missing values with 'THIRD'
US_combined.loc[:, 'PARTY'] = US_combined['PARTY'].fillna('THIRD')

# Create list of parties to rename
print(sorted(US_combined['PARTY'].unique()))

In [ ]:
# Define other parties to replace with 'THIRD' (remove DEMOCRAT and REPUBLICAN from 'US_combined' output)
other_parties = [
    'ALLIANCE', 'ALLIANCE PARTY', 'AMERICAN', 'AMERICAN CONSTITUTION', 'AMERICAN SHOPPING', 'AMERICAN SOLIDARITY', 'APPROVAL VOTING', 'BECOMING ONE NATION', 'BIRTHDAY', 'BLANK', 'BOILING FROG', 'BREAD AND ROSES', 'BULL MOOSE', 'C.U.P', 'CONSTITUTION', 'CONSTITUTION PARTY', 'CUP', 'FREEDOM AND PROSPERITY', 'GENEALOGY KNOW YOUR FAMILY HISTORY', 'GREEN', 'GREEN INDEPENDENT', 'GREEN-RAINBOW', 'GRUMPY OLD PATRIOTS', 'INDEPENDENCE', 'INDEPENDENCE ALLIANCE', 'INDEPENDENT', 'INDEPENDENT AMERICAN', 'LIBERTARIAN', 'LIBERTY UNION', 'LIFE', 'LIFE LIBERTY CONSTITUTION', 'NATURAL LAW PARTY', 'NONE', 'NONPARTISAN', 'OREGON PROGRESSIVE', 'OTHER', 'PACIFIC GREEN', 'PARTY FOR SOCIALISM AND LIBERATION', 'PROGRESSIVE', 'PROHIBITION', 'REFORM', 'SOCIALISM', 'SOCIALISM AND LIBERATION', 'SOCIALIST', 'SOCIALIST EQUALITY', 'SOCIALIST WORKERS', 'STATEWIDE GREEN', 'UNAFFILIATED', 'UNITY', 'UNITY AMERICA', 'UNITY OF COLORADO', 'US TAXPAYERS PARTY']

# Replace these other parties with 'THIRD'
US_combined['PARTY'] = US_combined['PARTY'].replace(other_parties, 'THIRD')

# Tally Presidential votes
PRES_votes = (US_combined.groupby('PARTY', as_index=False)['VOTES']
    .sum().sort_values(by='VOTES', ascending=False))

# Confirm
print(US_combined.info())
print(US_combined['PARTY'].unique())
print(US_combined['PARTY'].value_counts(dropna=False))
print(PRES_votes)

In [ ]:
# Pivot to get vote counts by Party
US_transform = (US_combined.groupby(['GEOID', 'PARTY'])['VOTES']
    .sum()
    .unstack(fill_value=0)
    .reset_index())

# Rename columns that were the party names after unstacking
US_transform = US_transform.rename(columns={
    'DEMOCRAT': 'DEM_VOTES',
    'REPUBLICAN': 'REP_VOTES',
    'THIRD': 'THRD_VOTES'})

# Change vote columns to int32
vote_cols = ['DEM_VOTES', 'REP_VOTES', 'THRD_VOTES']
US_transform[vote_cols] = US_transform[vote_cols].astype('int32')

# Confirm
#print(US_transform.info())
#print(US_transform.head())

## Create share of vote feature

In [ ]:
# Create new working dataframe
US_tranfm2 = US_transform.copy()

# Compute TOTAL_VOTES, drop any where the sum of all votes = 0
US_tranfm2['TOTAL_VOTES'] = US_tranfm2[vote_cols].sum(axis=1).astype('int32')
US_tranfm2 = US_tranfm2[US_tranfm2['TOTAL_VOTES'] != 0]

# Compute shares of votes
US_tranfm2['DEM_SHARE'] = (
    (US_tranfm2['DEM_VOTES'] / US_tranfm2['TOTAL_VOTES'])* 100).round(2)
US_tranfm2['REP_SHARE'] = (
    (US_tranfm2['REP_VOTES'] / US_tranfm2['TOTAL_VOTES'])* 100).round(2)
US_tranfm2['THRD_SHARE'] = (
    (US_tranfm2['THRD_VOTES'] / US_tranfm2['TOTAL_VOTES'])* 100).round(2)

# Confirm
#print(US_tranfm2.info())
#print(US_tranfm2.head())

## Create political leaning feature

In [ ]:
# View values of DEM_SHARE, ensure all >0
print(sorted(US_tranfm2['DEM_SHARE'].unique()))

In [ ]:
# Create new working dataframe
US_tranfm3 = US_tranfm2.copy()

# Define the political leaning function
def determine_win(row):
# Only DEM or REP win, only consider their shares for determining lead
    shares = {
        'DEM': row['DEM_SHARE'],
        'REP': row['REP_SHARE']}

    # Determine the winning party between DEM and REP
    if shares['DEM'] > shares['REP']:
        party_win = 1 # Democrat wins = Positive lead for DEM
        party_lead = (shares['DEM'] - shares['REP']) / 100
    elif shares['REP'] > shares['DEM']: # Corrected from else
        party_win = 0 # Republican wins = Negative lead for REP
        party_lead = (shares['DEM'] - shares['REP']) / 100
    else: # Tie (very unlikely)
        party_win = 2
        party_lead = 0.0

    return party_win, round(party_lead, 2)

# Apply function and create two new variables
US_tranfm3[['PARTY_WIN', 'PARTY_LEAD']] = US_tranfm3.apply(
    determine_win, axis=1).apply(pd.Series)

# Convert 'PARTY_WIN' to int
US_tranfm3['PARTY_WIN'] = US_tranfm3['PARTY_WIN'].astype('int32')

# Confirm
print(US_tranfm3.head(10))

## Save MEDSL data

In [ ]:
# Order variables
final_cols = ['GEOID', 'TOTAL_VOTES',
              'DEM_VOTES', 'DEM_SHARE',
              'REP_VOTES', 'REP_SHARE',
              'THRD_VOTES', 'THRD_SHARE',
              'PARTY_WIN', 'PARTY_LEAD']

# Create the tidy dataframe
MEDSL_tidy = US_tranfm3[final_cols]

MEDSL_tidy.to_csv('MEDSL_tidy.csv', index=False)

# Confirm
print(MEDSL_tidy.info())

# Merge data files

# Import _tidy files here if you do not want to clean the data

In [ ]:
import pandas as pd
# Import here if utilizing the _tidy files
DHC_tidy = pd.read_csv('DHC_tidy.csv')
PUR_tidy = pd.read_csv('PUR_tidy.csv')
MEDSL_tidy = pd.read_csv('MEDSL_tidy.csv')

# Confirm
print(DHC_tidy.info())
print(PUR_tidy.info())
print(MEDSL_tidy.info())

In [ ]:
# Merge first two files
TWO_join = pd.merge(DHC_tidy, PUR_tidy, on='GEOID', how='outer')

# Confirm
#print(TWO_join.info())

In [ ]:
# Merge with third dataset
FULL_DF = pd.merge(TWO_join, MEDSL_tidy, on='GEOID', how='outer')

# change GEOID type
FULL_DF['GEOID'] = FULL_DF['GEOID'].astype(str)

# Confirm
#print(FULL_DF.info())

In [ ]:
# Create new working dataframe
FULL_transform = FULL_DF.copy()

# Split 'Name' into 'County' and 'State'
FULL_transform[['County', 'State']] = FULL_transform[
    'County'].str.split(', ', expand=True)

# Reorder columns to move 'State' to index 1
cols = FULL_transform.columns.tolist()
cols.insert(1, cols.pop(cols.index('State')))
MERGED_DF = FULL_transform[cols]

## Save MERGED datafile

In [ ]:
MERGED_DF.to_csv('MERGED_DF.csv', index=False)

# Exploratory Data Analysis (with MERGED_DF)

In [ ]:
# Setup exploration environment
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
%matplotlib inline
print('Environment Ready')

In [ ]:
MERGED_DF = pd.read_csv('MERGED_DF.csv')
# ensure GEOID is an object
MERGED_DF['GEOID'] = MERGED_DF['GEOID'].astype(str)

# Confirm
print(MERGED_DF.info())

## Basic Information

In [ ]:
# Display descriptive statistics for numerical columns
print('Number of rows:', MERGED_DF.shape[0], '(Number of counties)')
print('Number of columns:', MERGED_DF.shape[1])
print('\nMissing Values: None')
print(MERGED_DF.isna().sum().sort_values(ascending=False))

print('\nDescriptive Statistics for Numerical Columns:')
display(MERGED_DF.describe())

# Display value counts for categorical column (PARTY_WIN)
print('\nValue Counts for 'PARTY_WIN' \n0: Republican Win\n1: Democrat Win:')
display(MERGED_DF['PARTY_WIN'].value_counts())

## Visualizations

In [ ]:
# Visualize the distribution of the target variable 'PARTY_WIN'
plt.figure(figsize=(6, 4))
sns.countplot(x='PARTY_WIN', data=MERGED_DF)
plt.title('Distribution of PARTY_WIN (0: Republican Win, 1: Democrat Win)')
plt.xlabel('Party Win')
plt.ylabel('Count')
plt.xticks([0, 1], ['Republican Win', 'Democrat Win'])
plt.show()

print('')
# Visualize the distribution of 'PARTY_LEAD'
fig, axes = plt.subplots(1, 3, figsize=(15,5))
sns.histplot(MERGED_DF['DEM_SHARE'], bins=30, kde=True, ax=axes[0], color='blue')
axes[0].set_title('Democratic Vote Share')
sns.histplot(MERGED_DF['REP_SHARE'], bins=30, kde=True, ax=axes[1], color='red')
axes[1].set_title('Republican Vote Share')
sns.histplot(MERGED_DF['PARTY_LEAD'], bins=30, kde=True, ax=axes[2], color='purple')
axes[2].set_title('Margin of Victory (Party Lead)')
plt.tight_layout()
plt.show()

print('')
sns.histplot(MERGED_DF['Pop_total'], bins=50, kde=True)
plt.title('County Population Distribution')
plt.xlabel('Population')
plt.ylabel('Number of Counties')
plt.show()

print('')
sns.histplot(MERGED_DF['%Urban_pop'], bins=30, kde=True)
plt.title('Urban Population Share by County')
plt.xlabel('% Urban Population')
plt.ylabel('Number of Counties')
plt.show()

print('')
sns.histplot(MERGED_DF['MED_AGE'], bins=30, kde=True)
plt.title('Median Age Distribution')
plt.xlabel('Median Age')
plt.ylabel('Number of Counties')
plt.show()

print('')
race_cols = ['%RACE_White', '%RACE_Black', '%RACE_Latino', '%RACE_Asian']
MERGED_DF[race_cols].plot(kind='box', figsize=(8,6))
plt.title('Distribution of Racial Composition by County')
plt.ylabel('Percentage')
plt.show()

print('')
sns.scatterplot(x='%OWN_HOME', y='%RENT_HOME', data=MERGED_DF)
plt.title('Own vs Rent in Counties')
plt.xlabel('% Own Home')
plt.ylabel('% Rent Home')
plt.show()

print('')
sns.scatterplot(x='%Urban_pop', y='DEM_SHARE', data=MERGED_DF, alpha=0.6)
plt.title('Urban Population vs Democratic Vote Share')
plt.show()

print('')
sns.scatterplot(x='%RACE_White', y='REP_SHARE', data=MERGED_DF, alpha=0.6, color='red')
plt.title('% White Population vs Republican Vote Share')
plt.show()

print('')
sns.scatterplot(x='MED_AGE', y='REP_SHARE', data=MERGED_DF, alpha=0.6, color='green')
plt.title('Median Age vs Republican Vote Share')
plt.show()

## Correlation checks on separated groups of features

In [ ]:
#corr_vars = ['Pop_total', 'MED_AGE', '%Urban_pop',
#             '%RACE_White', '%RACE_Black', '%RACE_Latino',
#             '%OWN_HOME', '%RENT_HOME',
#             'DEM_SHARE', 'REP_SHARE', 'PARTY_LEAD']

#corr = MERGED_DF[corr_vars].corr()
MERGED_num = MERGED_DF.select_dtypes(include=np.number)

corr = MERGED_num.corr()

plt.figure(figsize=(10,8))
sns.heatmap(corr, annot=False, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Correlation Heatmap')
plt.show()

### Analyze Age data

In [ ]:
# Create new working dataframe
MERGED_trform = MERGED_DF.copy()

# Define column groups for total, male, and female age percentages
age_total_cols = [
    col for col in MERGED_trform.columns if col.startswith('%TOTAL_')]
age_male_cols  = [
    col for col in MERGED_trform.columns if col.startswith('%MALE_')]
age_female_cols = [
    col for col in MERGED_trform.columns if col.startswith('%FEMALE_')]

# Combine all percentage age columns and the target variables
features_for_age = age_total_cols + age_male_cols + age_female_cols + [
    'PARTY_WIN', 'PARTY_LEAD']

# Calculate the correlation matrix for the selected features
corr_age = MERGED_trform[features_for_age].corr()

# Select and display only the correlations with PARTY_WIN and PARTY_LEAD
corr_age_subset = corr_age[['PARTY_WIN', 'PARTY_LEAD']].loc[
    age_total_cols + age_male_cols + age_female_cols]

# Plot heatmap for better visualization of correlations
plt.figure(figsize=(10, 15)) # Adjust figure size as needed
sns.heatmap(corr_age_subset,
            cmap='seismic_r',
            annot=True, fmt='.2f',
            vmin=-1, vmax=1)
plt.title('Correlation Heatmap: Percentage Age Groups vs PARTY_WIN and PARTY_LEAD')
plt.yticks(rotation=0)
plt.show()

#### With a clear divergence around age 55, compare 2 vs 3 age groupings  
- yng, mid, old: Looks to break groups into pos, neutral (between -0.1 and 0.1), neg  
- young, older: Looks to break age groups into positive and negative only  

In [ ]:
# Define lists of age columns for young, middle (cutoff is |0.1|), and old
age_male_yng = [col for col in MERGED_trform.columns if col.startswith('%MALE_') and any(age in col for age in ['18_19', '20_24', '25_29', '30_34', '35_39'])]
age_male_mid = [col for col in MERGED_trform.columns if col.startswith('%MALE_') and any(age in col for age in ['40_44', '45_49', '50_54'])]
age_male_old = [col for col in MERGED_trform.columns if col.startswith('%MALE_') and any(age in col for age in ['55_59', '60_64', '65_69', '70_74', '75_79', '80_84', '85+'])]
age_female_yng = [col for col in MERGED_trform.columns if col.startswith('%FEMALE_') and any(age in col for age in ['18_19', '20_24', '25_29', '30_34', '35_39', '40_44'])]
age_female_mid = [col for col in MERGED_trform.columns if col.startswith('%FEMALE_') and any(age in col for age in ['45_49', '50_54'])]
age_female_old = [col for col in MERGED_trform.columns if col.startswith('%FEMALE_') and any(age in col for age in ['55_59', '60_64', '65_69', '70_74', '75_79', '80_84', '85+'])]

# Define lists of age columns for young (cutoff is 0) and older
age_male_young = [col for col in MERGED_trform.columns if col.startswith('%MALE_') and any(age in col for age in ['18_19', '20_24', '25_29', '30_34', '35_39', '40_44', '45_49'])]
age_male_older = [col for col in MERGED_trform.columns if col.startswith('%MALE_') and any(age in col for age in ['50_54', '55_59', '60_64', '65_69', '70_74', '75_79', '80_84', '85+'])]
age_female_young = [col for col in MERGED_trform.columns if col.startswith('%FEMALE_') and any(age in col for age in ['18_19', '20_24', '25_29', '30_34', '35_39', '40_44', '45_49', '50_54'])]
age_female_older = [col for col in MERGED_trform.columns if col.startswith('%FEMALE_') and any(age in col for age in ['55_59', '60_64', '65_69', '70_74', '75_79', '80_84', '85+'])]

# Calculate the new aggregated percentage age groups
MERGED_trform['%AGE_MALE_YNG'] = MERGED_trform[age_male_yng].sum(axis=1).round(2)
MERGED_trform['%AGE_MALE_MID'] = MERGED_trform[age_male_mid].sum(axis=1).round(2)
MERGED_trform['%AGE_MALE_OLD'] = MERGED_trform[age_male_old].sum(axis=1).round(2)
MERGED_trform['%AGE_MALE_YOUNG'] = MERGED_trform[age_male_young].sum(axis=1).round(2)
MERGED_trform['%AGE_MALE_OLDER'] = MERGED_trform[age_male_older].sum(axis=1).round(2)

MERGED_trform['%AGE_FEMALE_YNG'] = MERGED_trform[age_female_yng].sum(axis=1).round(2)
MERGED_trform['%AGE_FEMALE_MID'] = MERGED_trform[age_female_mid].sum(axis=1).round(2)
MERGED_trform['%AGE_FEMALE_OLD'] = MERGED_trform[age_female_old].sum(axis=1).round(2)
MERGED_trform['%AGE_FEMALE_YOUNG'] = MERGED_trform[age_female_young].sum(axis=1).round(2)
MERGED_trform['%AGE_FEMALE_OLDER'] = MERGED_trform[age_female_older].sum(axis=1).round(2)

# Confirm
print(MERGED_trform.info())
print(MERGED_trform.head())

### Analyze race groups

In [ ]:
# Define the list of race percentage columns
race_cols = [col for col in MERGED_trform.columns if col.startswith('%RACE_')]

# Combine race percentage columns and the target variables
features_for_race = race_cols + ['PARTY_WIN', 'PARTY_LEAD']

# Calculate the correlation matrix
corr_race = MERGED_trform[features_for_race].corr()

# Select and display only the correlations with PARTY_WIN and PARTY_LEAD
corr_race_subset = corr_race[['PARTY_WIN', 'PARTY_LEAD']].loc[race_cols]

# Display the correlations
print('Correlation of Race/Ethnic Group Percentages with PARTY_WIN and PARTY_LEAD:')
display(corr_race_subset.sort_values(by='PARTY_LEAD', key=abs, ascending=False))

# Plot heatmap for better visualization
plt.figure(figsize=(8, 6))
sns.heatmap(corr_race_subset, cmap='seismic_r', annot=True, fmt='.2f', vmin=-1, vmax=1)
plt.title('Correlation Heatmap: Race/Ethnic Group Percentages vs PARTY_WIN and PARTY_LEAD')
plt.yticks(rotation=0)
plt.show()

With clear racial differences, I will try two variations of race groups  
- White and Non-White  
- White, strong political lean, more neutral lean

In [ ]:
# Define lists to compare 2 groups: non-whites or with a cutoff of |0.2|
RACE_NonWhite = [col for col in MERGED_trform.columns if col.startswith('%RACE_') and any(race in col for race in ['Asian', 'Black', 'Other', 'Latino', 'Native', 'HI_PI', 'Mixed'])]
RACE_BAO = [col for col in MERGED_trform.columns if col.startswith('%RACE_') and any(race in col for race in ['Black', 'Asian', 'Other'])]
RACE_LNHM = [col for col in MERGED_trform.columns if col.startswith('%RACE_') and any(race in col for race in ['Latino', 'Native', 'HI_PI', 'Mixed'])]

# Calculate the new aggregated percentage race groups
MERGED_trform['%RACE_NonWhite'] = MERGED_trform[RACE_NonWhite].sum(axis=1).round(2)
MERGED_trform['%RACE_BAO'] = MERGED_trform[RACE_BAO].sum(axis=1).round(2)
MERGED_trform['%RACE_LNHM'] = MERGED_trform[RACE_LNHM].sum(axis=1).round(2)

### Analyze relationship groups

In [ ]:
# Filter for columns starting with '%REL_'
rel_cols = [col for col in MERGED_trform.columns if col.startswith('%REL_')]

# Calculate the correlation of these columns with PARTY_WIN and PARTY_LEAD
corr_rel = MERGED_trform[rel_cols + ['PARTY_WIN', 'PARTY_LEAD']].corr()

# Select and display only the correlations with PARTY_WIN and PARTY_LEAD
corr_rel_subset = corr_rel[['PARTY_WIN', 'PARTY_LEAD']].loc[rel_cols]

# Display the correlations
print('Correlation of Relationship Variables with PARTY_WIN and PARTY_LEAD:')
display(corr_rel_subset.sort_values(by='PARTY_LEAD', key=abs, ascending=False))

# Optional: Visualize correlations as a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(corr_rel_subset,
            cmap='seismic_r',
            annot=True, fmt='.2f',
            vmin=-1, vmax=1)
plt.title('Correlation Heatmap: Relationship Variables vs PARTY_WIN and PARTY_LEAD')
plt.yticks(rotation=0)
plt.show()

### Analyze household groups

In [ ]:
# Filter for columns starting with '%HH_'
hh_cols = [col for col in MERGED_trform.columns if col.startswith('%HH_')]

# Add own and urban columns
hh_cols.extend(['%OWN_HOME', '%Urban_pop'])

# Correlate these columns with PARTY_WIN and PARTY_LEAD
corr_hh = MERGED_trform[hh_cols + ['PARTY_WIN', 'PARTY_LEAD']].corr()

# Select and display only the correlations with PARTY_WIN and PARTY_LEAD
corr_hh_subset = corr_hh[['PARTY_WIN', 'PARTY_LEAD']].loc[hh_cols]

# Display the correlations
print('Correlation of Household, Ownership, and Urban Variables with PARTY_WIN and PARTY_LEAD:')
display(corr_hh_subset.sort_values(by='PARTY_LEAD', key=abs, ascending=False))

# Optional: Visualize correlations as a heatmap
plt.figure(figsize=(8, 10)) # Adjusted figure size
sns.heatmap(corr_hh_subset,
            cmap='seismic_r',
            annot=True, fmt='.2f',
            vmin=-1, vmax=1)
plt.title('Correlation Heatmap: Household, Ownership, and Urban Variables vs PARTY_WIN and PARTY_LEAD')
plt.yticks(rotation=0)
plt.show()

## Save VOTE_DF

In [ ]:
# List columns to keep (drop HH_totals, only keep M_total and F_total as ref)
columns_to_keep = [
    'GEOID', 'Male_total', 'Female_total', '%AGE_MALE_YNG', '%AGE_MALE_MID', '%AGE_MALE_OLD', '%AGE_MALE_YOUNG', '%AGE_MALE_OLDER', '%AGE_FEMALE_YNG', '%AGE_FEMALE_MID', '%AGE_FEMALE_OLD', '%AGE_FEMALE_YOUNG', '%AGE_FEMALE_OLDER', '%RACE_White', '%RACE_Black', '%RACE_Latino', '%RACE_Native', '%RACE_Asian', '%RACE_HI_PI', '%RACE_Other', '%RACE_Mixed', '%RACE_NonWhite', '%RACE_BAO', '%RACE_LNHM', '%REL_OP_SEX_MAR', '%REL_OP_SEX_UNMAR', '%REL_S_SEX_MAR', '%REL_S_SEX_UNMAR', '%REL_W_RELATIVES', '%REL_NON_REL', '%REL_MALE_JAILED', '%REL_FEMALE_JAILED', '%REL_MALE_GRP_DORM', '%REL_FEMALE_GRP_DORM', '%HH_MARRIED', '%HH_MAR_W_KIDS','%HH_NOT_MAR',  '%HH_NOT_MAR_W_KIDS', '%HH_MALE_ALONE', '%HH_MALE_65+', '%HH_MALE_W_KIDS', '%HH_FEMALE_ALONE', '%HH_FEMALE_65+', '%HH_FEMALE_W_KIDS', '%OWN_HOME', '%Urban_pop', 'PARTY_WIN', 'PARTY_LEAD']

# Create VOTE dataframe
VOTE_DF = MERGED_trform[columns_to_keep].copy()

VOTE_DF.to_csv('VOTE_DF.csv', index=False)

## EDA complete; dataframe cleaned, merged, transformed, partially reduced, and ready for analysis

# Feature analysis (with VOTE_DF)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import numpy.typing as npt
import statsmodels.api as sm
from typing import Literal, Tuple, Union
from scipy.stats import shapiro, mannwhitneyu, rankdata, norm
from sklearn.linear_model import LogisticRegression, LassoCV
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, mean_squared_error, r2_score, mean_absolute_error, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.inspection import permutation_importance, PartialDependenceDisplay
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.manova import MANOVA
from collections import Counter
%matplotlib inline
print('Environment Ready')

# Import VOTE_DF file here for analysis of features
(Looked at feature interactions as well, but opted to keep simple as few improved modeling)

In [ ]:
VOTE_DF = pd.read_csv('VOTE_DF.csv')
# ensure GEOID is an object
VOTE_DF['GEOID'] = VOTE_DF['GEOID'].astype(str)

# Inspect
#print(VOTE_DF.info())

## Variance check

In [ ]:
# Select numerical columns
VOTE_num = VOTE_DF.select_dtypes(include=np.number)

variances = VOTE_num.var()

# Sort variances in descending order
var_sorted = variances.sort_values(ascending=True)

# Set pandas display option to show float format
pd.set_option('display.float_format', '{:.2f}'.format)

# Confirm (Consider dropping features with low variance >0.05)
print('\nFeature Variances (sorted):')
print(var_sorted.head(20))

## Compute VIF for VOTE_DF

In [ ]:
# Remove independent variables
VOTE_features = VOTE_num.drop(columns=['PARTY_WIN', 'PARTY_LEAD'])

# Add required constant
VOTE_features = sm.add_constant(VOTE_features)

# Compute Variance Inflation Factor for each feature
VOTE_VIF = pd.DataFrame()
VOTE_VIF['Feature'] = VOTE_features.columns
# Compute VIF, handling potential inf values which occur with perfect multicollinearity
VOTE_VIF['VIF'] = [variance_inflation_factor(VOTE_features.values, i) for i in range(VOTE_features.shape[1])]

# Sort by VIF in descending order for easier analysis
VOTE_VIF = VOTE_VIF.sort_values(by='VIF', ascending=False)

# Set display to show float format
pd.set_option('display.float_format', '{:.2f}'.format)

print('VIF for VOTE_DF:')
display(VOTE_VIF)

# Correlations

## Pearson Correlation Matrix

In [ ]:
# Compute Pearson correlation matrix
pearson_corr_matrix = VOTE_DF.corr(method='pearson')

# Display the correlations
print('Pearson Correlation Matrix:')
display(pearson_corr_matrix)

# Sort Pearson correlations with PARTY_WIN and PARTY_LEAD
pearson_corr_win = pearson_corr_matrix['PARTY_WIN'].sort_values(ascending=False)
pearson_corr_lead = pearson_corr_matrix['PARTY_LEAD'].sort_values(ascending=False)

## Spearman Correlation Matrix

In [ ]:
# Compute Spearman correlation matrix
spearman_corr_matrix = VOTE_DF.corr(method='spearman')

# Display the correlations
print('\nSpearman Correlation Matrix:')
display(spearman_corr_matrix)

# Sort and store Spearman correlation results
spearman_corr_win = spearman_corr_matrix['PARTY_WIN'].sort_values(ascending=False)
spearman_corr_lead = spearman_corr_matrix['PARTY_LEAD'].sort_values(ascending=False)

## Chatterjee's Correlation  
In 2020, a paper titled 'A New Coefficient of Correlation' introduced a new coefficient measure ξ (“Xi”) which measures how much the dependent variable is a function of the independent. The result equals 0 if the two variables are independent and will be closer to 1 as the relationship strengthens. Also includes some theoretical properties that allow for hypothesis testing prior to making assumptions about the data.  

Along with the article, the R package 'XICOR' was released which contains the function xicor() which calculates ξ when X and Y vectors or matrices are provided (provides p-values for hypothesis testing).

S. Chatterjee, *A New Coefficient of Correlation* (2020), Journal of the American Statistical Association.
https://doi.org/10.48550/arXiv.1909.10140

The below code is a python xicor function based on one written by Tim Sumner https://medium.com/data-science/a-new-coefficient-of-correlation-64ae4f260310

In [ ]:
# Compute Chatterjee's Correlation
def xicor(X, Y, ties='auto', return_p=True):
    np.random.seed(1)
    X = np.asarray(X)
    Y = np.asarray(Y)
    Y_sorted = Y[np.argsort(X)]
    n = len(X)

    if ties == 'auto':
        ties = len(np.unique(Y)) < n

    if ties:
        r = rankdata(Y_sorted, method='ordinal')
        l = rankdata(Y_sorted, method='max')
        xi = 1 - n * np.sum(np.abs(np.diff(r))) / (2 * np.sum(l * (n - l)))
    else:
        r = rankdata(Y_sorted, method='ordinal')
        xi = 1 - 3 * np.sum(np.abs(np.diff(r))) / (n**2 - 1)

# p-value approximation
    p_value = norm.sf(xi, scale=2/5/np.sqrt(n))

    if return_p:
        return xi, p_value
    else:
        return xi

# Define the independent and dependent variables
features = [col for col in VOTE_DF.columns if col not in [
    'PARTY_WIN', 'PARTY_LEAD',
    'Male_total', 'Female_total']]

target_win = VOTE_DF['PARTY_WIN']
target_lead = VOTE_DF['PARTY_LEAD']

# Store xicor results
xicor_results_win = {}
xicor_results_lead = {}

# Compute xicor for each feature against PARTY_WIN
for feature in features:
    x_data = VOTE_DF[feature]
    xi_stat, xi_p_value = xicor(x_data, target_win)
    xicor_results_win[feature] = {'statistic': xi_stat, 'p_value': xi_p_value}
    #print(f'{feature}: Statistic={xi_stat:.2f}, P-value={xi_p_value:.2f}')

# Compute xicor for each feature against PARTY_LEAD
for feature in features:
    x_data = VOTE_DF[feature]
    xi_stat, xi_p_value = xicor(x_data, target_lead)
    xicor_results_lead[feature] = {'statistic': xi_stat, 'p_value': xi_p_value}
    #print(f'{feature}: Statistic={xi_stat:.2f}, P-value={xi_p_value:.2f}')

# Store Chatterjee correlation results
xi_corr_win = pd.DataFrame.from_dict(xicor_results_win, orient='index')
xi_corr_lead = pd.DataFrame.from_dict(xicor_results_lead, orient='index')

## Compare Correlation Coefficients

In [ ]:
# Combine all correlation results into a single DataFrame
correlation_comparison = pd.concat([
    xi_corr_lead['statistic'].rename('Xi_Corr_LEAD'),
    xi_corr_win['statistic'].rename('Xi_Corr_WIN'),
    pearson_corr_lead,
    pearson_corr_win,
    spearman_corr_lead,
    spearman_corr_win,
], axis=1)

# Remove the target variables  if included
correlation_comparison.drop(['PARTY_WIN', 'PARTY_LEAD'], errors='ignore', inplace=True)

# Rename features
Correlation_Table = correlation_comparison.rename(columns={
    'Pearson_Corr_PARTY_LEAD': 'Pearson_LEAD',
    'Pearson_Corr_PARTY_WIN': 'Pearson_WIN',
    'Spearman_Corr_PARTY_LEAD': 'Spearman_LEAD',
    'Spearman_Corr_PARTY_WIN': 'Spearman_WIN'})

# Display all correlations
print('Comparison of Xi, Pearson, and Spearman Correlations:')
display(Correlation_Table.round(4).sort_values(by='Xi_Corr_LEAD', ascending=False))

# Statistical test (Test for normality first)  


In [ ]:
# Separate the dataframe into two groups based on PARTY_WIN
group_Republican = VOTE_num[VOTE_num['PARTY_WIN'] == 0]
group_Democrat = VOTE_num[VOTE_num['PARTY_WIN'] == 1]

features_for_norm = VOTE_num.columns.tolist()
features_for_norm.remove('PARTY_WIN')

normality_results = {}

for feature in features_for_norm:
    data1 = group_Republican[feature]
    data2 = group_Democrat[feature]

    if len(data1) > 2 and len(data2) > 2:
        stat1, p_norm1 = shapiro(data1)
        stat2, p_norm2 = shapiro(data2)

        normality_results[feature] = {
            'Rep_p': f'{p_norm1:.2f}',
            'Dem_p': f'{p_norm2:.2f}'}
    else:
        normality_results[feature] = {
            'Rep_p': None,
            'Dem_p': None}

# Convert to DataFrame
normality_df = pd.DataFrame(normality_results).T

# Confirm (Normality will be defined as above a threshhold of 0.05)
print(normality_df)

> Almost every feature is way below 0.05 in both groups: normality is violated with one exception: Will not use T-test.

## Run Mann-Whitney U Test

In [ ]:
mannwhit_results = []

for feature in features_for_norm:
    if feature == 'PARTY_LEAD':
        continue

    data1 = group_Republican[feature]
    data2 = group_Democrat[feature]

    if len(data1) < 2 or len(data2) < 2:
        continue

    U_stat, p_value = mannwhitneyu(data1, data2, alternative='two-sided')

    if p_value < 0.05:
        mannwhit_results.append({
            'Feature': feature,
            'DEM_median': data2.median(),
            'REP_median': data1.median(),
            'U_stat': U_stat,
            'p_value': p_value,
            'n_dem': len(data2),
            'n_rep': len(data1)})

mannwhit_df = pd.DataFrame(mannwhit_results)

# Derive additional stats
mannwhit_df['diff_median'] = mannwhit_df['DEM_median'] - mannwhit_df['REP_median']

mannwhit_df['R_biserial'] = 1 - (2 * mannwhit_df['U_stat'] / (
                            mannwhit_df['n_dem'] * mannwhit_df['n_rep']))

mannwhit_df['Cohens_d'] = (2 * mannwhit_df['R_biserial']
                          ) / np.sqrt(1 - mannwhit_df['R_biserial']**2)

# Add qualitative labels
def label_effect_size(d):
    d = abs(d)
    if d < 0.2:
        return 'Negligible'
    elif d < 0.5:
        return 'Small'
    elif d < 0.8:
        return 'Medium'
    else:
        return 'Large'

mannwhit_df['Effect_size'] = mannwhit_df['Cohens_d'].astype(float).apply(label_effect_size)

# Reorder columns for priority in table (consider dropping n_ features)
cols = mannwhit_df.columns.tolist()
cols.insert(3, cols.pop(cols.index('diff_median')))
cols.insert(5, cols.pop(cols.index('Cohens_d')))
cols.insert(6, cols.pop(cols.index('Effect_size')))
cols.insert(7, cols.pop(cols.index('R_biserial')))
mannwhit_df = mannwhit_df[cols]

# Format after sorting
mannwhit_df['DEM_median'] = mannwhit_df['DEM_median'].map(lambda x: f'{x:.2f}')
mannwhit_df['REP_median'] = mannwhit_df['REP_median'].map(lambda x: f'{x:.2f}')
mannwhit_df['diff_median'] = mannwhit_df['diff_median'].map(lambda x: f'{x:.2f}')
mannwhit_df['Cohens_d'] = mannwhit_df['Cohens_d'].map(lambda x: f'{x:.2f}')
mannwhit_df['R_biserial'] = mannwhit_df['R_biserial'].map(lambda x: f'{x:.2f}')
mannwhit_df['p_value'] = mannwhit_df['p_value'].map(lambda x: f'{x:.2f}')

# Confirm
display(mannwhit_df.sort_values(by='Cohens_d', ascending=False))

# Feature importance

## Feature Importance for PARTY_WIN from Logistic Regression

In [ ]:
# Define the features to exclude based on p-values
# Could drop Same_Sex features, but not ready to drop yet
features_to_exclude = ['']

# Select features for logistic regression, excluding the specified ones
features_for_logit = [col for col in VOTE_DF.columns if col not in features_to_exclude + ['GEOID', 'Male_total', 'Female_total', 'PARTY_WIN', 'PARTY_LEAD']]

X0 = VOTE_DF[features_for_logit]
y0 = VOTE_DF['PARTY_WIN']

# Split data into training and testing sets (recommended for model evaluation)
X0_train, X0_test, y0_train, y0_test = train_test_split(
    X0, y0, test_size=0.2, random_state=1,
    stratify=y0) # To maintain class distribution

# Standardize the features
scaler = StandardScaler()
X0_train_scaled = scaler.fit_transform(X0_train)
X0_test_scaled = scaler.transform(X0_test)

# Initialize and train the Logistic Regression model with regularization
# Using default L2 penalty and balanced class weight
logit_model_sklearn = LogisticRegression(
    random_state=1,
    class_weight='balanced',
    max_iter=1000) # Increased max_iter for convergence
logit_model_sklearn.fit(X0_train_scaled, y0_train)

# Confirm feature importances from the trained model (coefficients)
print('Feature Importance (Coefficients from Regularized Logistic Regression):')
logit_feature_importance = pd.Series(
    logit_model_sklearn.coef_[0], index=features_for_logit)
print(logit_feature_importance.sort_values(ascending=False))

# Plot feature importances
plt.figure(figsize=(10, 8)) # Adjusted figure size for better readability
logit_feature_importance.sort_values().plot(kind='barh')
plt.title('Feature Importance from Logistic Regression(WIN)')
plt.xlabel('Coefficient Value')
plt.ylabel('Feature')
plt.show()

# Evaluate the model on the test set
y0_pred_logit = logit_model_sklearn.predict(X0_test_scaled)

print('\nLogistic Regression Model Evaluation (on test set):')
print(f'Accuracy: {accuracy_score(y0_test, y0_pred_logit):.2f}')
print('Classification Report:')
print(classification_report(y0_test, y0_pred_logit))
print('Confusion Matrix:')
print(confusion_matrix(y0_test, y0_pred_logit))

## Feature Importance for PARTY_WIN from Decision Tree Classifier

In [ ]:
# Define the features (X) and the target variable (y)
# Exclude the target variables themselves from the features
features_for_dtc = [col for col in VOTE_DF.columns if col not in [
    'GEOID', 'Male_total', 'Female_total', 'PARTY_WIN', 'PARTY_LEAD']]

# Define the features (X) and the target variable (y)
X1 = VOTE_DF[features_for_dtc]
y1 = VOTE_DF['PARTY_WIN']

# Split data into training and testing sets
X1_train, X1_test, y1_train, y1_test = train_test_split(
    X1, y1, test_size=0.2, random_state=1)

# Initialize and train the Decision Tree Regressor model
dtc_model = DecisionTreeClassifier(
    random_state=1,
    max_depth=5,
    min_samples_split=20,
    min_samples_leaf=10)

dtc_model.fit(X1_train, y1_train)

# Make predictions on the test set
y1_pred_dtc = dtc_model.predict(X1_test)

# Evaluate the model
mse = mean_squared_error(y1_test, y1_pred_dtc)
rmse = np.sqrt(mse) # Calculate RMSE manually
mae = mean_absolute_error(y1_test, y1_pred_dtc)
r2 = r2_score(y1_test, y1_pred_dtc)

print('Decision Tree Regressor Model Evaluation (on test set):')
print(f'Mean Squared Error (MSE): {mse:.2f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')
print(f'Mean Absolute Error (MAE): {mae:.2f}')
print(f'R-squared (R2): {r2:.2f}')

# Plot feature importances from the trained model
print('\nFeature Importance from Decision Tree Regressor:')
dtc_feature_importance = pd.Series(dtc_model.feature_importances_, index=features_for_dtc)

# Sort and print feature importances
print(dtc_feature_importance.sort_values(ascending=False).head(15))

# Plot feature importances
plt.figure(figsize=(10, 8)) # Adjusted figure size
dtc_feature_importance.sort_values().plot(kind='barh')
plt.title('Feature Importance from Decision Tree Regressor(WIN)')
plt.xlabel('Importance Score')
plt.ylabel('Feature')
plt.show()

## Feature Importance for PARTY_LEAD from Decision Tree Regressor

In [ ]:
X2 = VOTE_DF[features_for_dtc]
y2 = VOTE_DF['PARTY_LEAD']

# Split data into training and testing sets
X2_train, X2_test, y2_train, y2_test = train_test_split(
    X2, y2, test_size=0.2, random_state=1)

# Initialize and train the Decision Tree Regressor model
dtr_model = DecisionTreeRegressor(
    random_state=1,
    max_depth=5,
    min_samples_split=20,
    min_samples_leaf=10)

dtr_model.fit(X2_train, y2_train)

# Make predictions on the test set
y2_pred_dtr = dtr_model.predict(X2_test)

# Evaluate the model
mse = mean_squared_error(y2_test, y2_pred_dtr)
rmse = np.sqrt(mse) # Calculate RMSE manually
mae = mean_absolute_error(y2_test, y2_pred_dtr)
r2 = r2_score(y2_test, y2_pred_dtr)

print('Decision Tree Regressor Model Evaluation (on test set):')
print(f'Mean Squared Error (MSE): {mse:.2f}')
print(f'Root Mean Squared Error (RMSE): {rmse:.2f}')
print(f'Mean Absolute Error (MAE): {mae:.2f}')
print(f'R-squared (R2): {r2:.2f}')

# Get and plot feature importances from the trained model
print('\nFeature Importance from Decision Tree Regressor:')
dtr_feature_importance = pd.Series(dtr_model.feature_importances_, index=features_for_dtc)

# Sort and print feature importances
print(dtr_feature_importance.sort_values(ascending=False).head(20))

# Plot feature importances
plt.figure(figsize=(10, 8)) # Adjusted figure size
dtr_feature_importance.sort_values().plot(kind='barh')
plt.title('Feature Importance from Decision Tree Regressor(LEAD)')
plt.xlabel('Importance Score')
plt.ylabel('Feature')
plt.show()

## Feature Importance for PARTY_WIN from Random Forest

In [ ]:
# Select features for the Random Forest model
# We can use the same set of features that worked for the logistic regression.
features_for_rf = features_for_logit

X3 = VOTE_DF[features_for_rf]
y3 = VOTE_DF['PARTY_WIN']

# Split data into training and testing sets
X3_train, X3_test, y3_train, y3_test = train_test_split(
    X3, y3, test_size=0.2, random_state=1, stratify=y3)

# Initialize and train the Random Forest Classifier
# Use a reasonable number of estimators (n_estimators) and a random state for reproducibility
rf_model = RandomForestClassifier(
    n_estimators=300,
    max_depth=None,        # control/avoid overfitting
    min_samples_split=10,  # avoid tiny splits
    min_samples_leaf=5,    # smoother trees
    random_state=1,
    class_weight='balanced')
rf_model.fit(X3_train, y3_train)

# Get feature importances from the trained model
rf_feature_importance = pd.Series(
    rf_model.feature_importances_, index=features_for_rf)

# Sort and print feature importances
print('Feature Importance from Random Forest:')
print(rf_feature_importance.sort_values(ascending=False))

# Plot feature importances
plt.figure(figsize=(10, 6))
rf_feature_importance.sort_values().plot(kind='barh')
plt.title('Feature Importance from Random Forest(WIN)')
plt.xlabel('Importance Score (Mean Decrease in Impurity)')
plt.ylabel('Feature')
plt.show()

# Evaluate the model on the test set
y3_pred_rf = rf_model.predict(X3_test)

print('\nRandom Forest Model Evaluation (on test set):')
print(f'Accuracy: {accuracy_score(y3_test, y3_pred_rf):.2f}')
print('Classification Report:')
print(classification_report(y3_test, y3_pred_rf))
print('Confusion Matrix:')
print(confusion_matrix(y3_test, y3_pred_rf))

### Define permutation importance function (with optional cross-validation)

In [ ]:
# Compute permutation importance (PI) or cross-validated PI (CV-PI)
def get_PI(model, X3, y3, cv=False, n_splits=5, n_repeats=10, random_state=1):
    '''
    Parameters
    ----------
    model : estimator
        Trained model (must support predict).
    X : DataFrame
        Features used for prediction.
    y : Series or array-like
        Target values.
    cv : bool, default=False
        If True, performs cross-validated permutation importance.
    n_splits : int, default=5
        Number of CV folds (only used if cv=True).
    n_repeats : int, default=10
        Number of shuffles for permutation importance.
    random_state : int, default=1
        Random seed for reproducibility.

    Returns
    -------
    importance_df : DataFrame
        Feature importances sorted by mean decrease in score.
    '''

    if not cv:
# Standard PI on a single fitted model
        result = permutation_importance(model, X3, y3,
                                        n_repeats=n_repeats,
                                        random_state=random_state,
                                        n_jobs=-1)
        importance_df = pd.DataFrame({
            'Feature': X3.columns,
            'Importance Mean': result.importances_mean,
            'Importance Std': result.importances_std
        }).sort_values(by='Importance Mean', ascending=False)

    else:
# Cross-validated PI
        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
        importances = []

        for train_idx, test_idx in skf.split(X3, y3):
            X3_train, X3_test = X3.iloc[train_idx], X3.iloc[test_idx]
            y3_train, y3_test = y3.iloc[train_idx], y3.iloc[test_idx]

            model.fit(X3_train, y3_train)
            result = permutation_importance(model, X3_test, y3_test,
                                            n_repeats=n_repeats,
                                            random_state=random_state,
                                            n_jobs=-1)
            importances.append(result.importances_mean)

        mean_importances = np.mean(importances, axis=0)
        std_importances = np.std(importances, axis=0)

        importance_df = pd.DataFrame({
            'Feature': X3.columns,
            'Importance Mean': mean_importances,
            'Importance Std': std_importances
        }).sort_values(by='Importance Mean', ascending=False)
    return importance_df

# Confirm
pd.set_option('display.max_rows', None)
pd.reset_option('display.float_format')

RF_PI = get_PI(rf_model, X3_test, y3_test, cv=False)
print(RF_PI)

# Plot permutation importances
plt.figure(figsize=(10, 6))
# Convert 'Importance Mean' to numeric before plotting
RF_PI['Importance Mean'] = pd.to_numeric(RF_PI['Importance Mean'])
RF_PI.sort_values(by='Importance Mean', ascending=True).plot(kind='barh')
plt.title('Permutation Importance from Random Forest(WIN)')
plt.xlabel('Importance Score (Importance decrease in Mean)')
plt.ylabel('Feature')
plt.show()

### Run RFECV with Random Forest to confirm best features

In [ ]:
# Utillize X, y, train, test from Logit (X0, y0)
# RFECV with Random Forest
rf = RandomForestClassifier(n_estimators=500, random_state=1, class_weight='balanced')

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
selector = RFECV(estimator=rf, step=1, cv=cv, scoring='accuracy', n_jobs=-1)
selector.fit(X0_train, y0_train)

# Best features
best_features = X0.columns[selector.support_].tolist()
print('Best feature subset:')
print(best_features)

# Retrain final model with best features
rf_ECV = RandomForestClassifier(n_estimators=500, random_state=1, class_weight='balanced')
rf_ECV.fit(X0_train[best_features], y0_train)
y_pred_ECV = rf_ECV.predict(X0_test[best_features])

print('\nFinal Model Evaluation with Best Features:')
print(f'Accuracy: {accuracy_score(y0_test, y_pred_ECV):.4f}')
print('Classification Report:')
print(classification_report(y0_test, y_pred_ECV))
print('Confusion Matrix:')
print(confusion_matrix(y0_test, y_pred_ECV))

## Feature importance for PARTY_LEAD from Lasso Regression after Cross-validate Alpha

In [ ]:
# Define features and exclude target variables
features_for_lasso_cv = [col for col in VOTE_DF.columns if col not in ['GEOID', 'PARTY_WIN', 'PARTY_LEAD']]
X4 = VOTE_DF[features_for_lasso_cv]
y4 = VOTE_DF['PARTY_LEAD']

# Split into train and test sets
X4_train, X4_test, y4_train, y4_test = train_test_split(
    X4, y4, test_size=0.2, random_state=1)

# Standardize the features
scaler = StandardScaler()
X4_train_scaled = scaler.fit_transform(X4_train)
X4_test_scaled = scaler.transform(X4_test)

# Let LassoCV automatically generates an alpha grid to test
lasso_cv_model = LassoCV(cv=5, random_state=1, max_iter=10000)
lasso_cv_model.fit(X4_train_scaled, y4_train)

# Confirm the optimal alpha found by LassoCV
optimal_alpha = lasso_cv_model.alpha_
print(f'Optimal alpha found by LassoCV: {optimal_alpha:.4f}')
print('')
# Plot the MSE as a function of alpha
mse_path = lasso_cv_model.mse_path_
alphas = lasso_cv_model.alphas_

plt.figure(figsize=(10, 6))
plt.plot(alphas, mse_path, linestyle='-', marker='o')
plt.xscale('log') # Often useful to plot alpha on a log scale
plt.xlabel('Alpha')
plt.ylabel('Mean Squared Error (across folds)')
plt.title('Mean Squared Error vs. Alpha during Cross-validation')
plt.axvline(optimal_alpha, color='red', linestyle='--', label=f'Optimal Alpha = {optimal_alpha:.4f}')
plt.legend()
plt.grid(True)
plt.show()

# List the coefficients with the optimal alpha
print('\nFeature Importance from LassoCV (CV = 0.0002):')
feature_importance_lasso_cv = pd.Series(lasso_cv_model.coef_, index=features_for_lasso_cv)
print(feature_importance_lasso_cv.sort_values(ascending=False))

# Plot feature importances with optimal alpha
plt.figure(figsize=(10, 10))
feature_importance_lasso_cv.sort_values().plot(kind='barh')
plt.title(f'Feature Importance from Lasso Regression(LEAD) with Optimal Alpha = {optimal_alpha:.4f}')
plt.xlabel('Coefficient Value')
plt.ylabel('Feature')
plt.show()

# Evaluate the final Lasso model with the optimal alpha on the test set
y4_pred_lasso_cv = lasso_cv_model.predict(X4_test_scaled)

print('\nLasso Regression Model Evaluation (with Optimal Alpha):')
mse_test = mean_squared_error(y4_test, y4_pred_lasso_cv)
rmse_test = np.sqrt(mse_test)
r2_test = r2_score(y4_test, y4_pred_lasso_cv)

print(f'Mean Squared Error (MSE) on test set: {mse_test:.4f}')
print(f'Root Mean Squared Error (RMSE) on test set: {rmse_test:.4f}')
print(f'R-squared (R2) on test set: {r2_test:.4f}')

## Compare feature importance

In [ ]:
def normalize_series(s, keep_sign=True):
    '''Normalize feature importance to 0–1 scale, optionally keeping sign.'''
    s = s.fillna(0)
    if keep_sign:
        return s / s.abs().max()  # scale to -1..1, preserving sign
    else:
        scaler = MinMaxScaler()
        return pd.Series(scaler.fit_transform(s.values.reshape(-1, 1)).flatten(), index=s.index)

# Collect raw importances into a DataFrame
feature_importances = pd.DataFrame({
    'LogReg': logit_feature_importance,
    'DecTreeClass': dtc_feature_importance,
    'DecTreeReg': dtr_feature_importance,
    'RandomForest': rf_feature_importance,
    'RF_PI': RF_PI.set_index('Feature')['Importance Mean'],  # permutation importance
    'Lasso_LogReg': feature_importance_lasso_cv})

# Normalize each column (preserving signs)
for col in feature_importances.columns:
    if col in ['LogReg', 'Lasso_LogReg']:  # signed coefficients
        feature_importances[col] = normalize_series(feature_importances[col], keep_sign=True)
    else:  # tree-based importances are ≥ 0
        feature_importances[col] = normalize_series(feature_importances[col], keep_sign=False)

# Compute mean rank or average importance across models
feature_importances['Avg_Importance'] = feature_importances.abs().mean(axis=1)

# Sort by average importance
feature_importances = feature_importances.sort_values(by='Avg_Importance', ascending=False)

# Print the top features
print('\nTop 25 Features Across Models (normalized):')
display(feature_importances.round(4))

# RFECV Feature Selection for final model

In [ ]:
# Prepare Data, drop reference and overlap features
drop_features = [
    'Male_total', 'Female_total', # reference only
    '%AGE_MALE_YNG', '%AGE_MALE_MID', '%AGE_MALE_OLD', # overlap
    '%AGE_FEMALE_YNG', '%AGE_FEMALE_MID', '%AGE_FEMALE_OLD', # overlap
    '%RACE_NonWhite', '%RACE_BAO', '%RACE_LNHM'] # overlap

VOTE_FULL = VOTE_DF.drop(
    columns=drop_features,
    errors='ignore')

X = VOTE_FULL.drop(
    columns=['GEOID', 'PARTY_WIN', 'PARTY_LEAD'])
y = VOTE_FULL['PARTY_WIN']

# Train/test split
XF_train, XF_test, yF_train, yF_test = train_test_split(
    X, y, test_size=0.2, random_state=1, stratify=y)

# Recursive Feature Elimination with Cross-Validation
rf = RandomForestClassifier(
    n_estimators=400, random_state=1, class_weight='balanced')
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
selector = RFECV(
    estimator=rf, step=1, cv=cv, scoring='accuracy', n_jobs=-1)
selector.fit(XF_train, yF_train)

# Plot accuracy vs. number of features
n_features = np.arange(
    1, len(selector.cv_results_['mean_test_score']) + 1)

plt.figure(figsize=(8,6))
plt.plot(n_features, selector.cv_results_['mean_test_score'], marker='o')
plt.axhline(0.93, color='red', linestyle='--', label='93% threshold')
plt.axhline(max(selector.cv_results_['mean_test_score']), color='green', linestyle='--', label='Best Acc')
plt.xlabel('Number of Features Selected')
plt.ylabel('Cross-Validated Accuracy')
plt.title('Accuracy vs. Number of Features (RFECV)')
plt.legend()
plt.show()

# Retrain with Best Features
best_features = X.columns[
    selector.support_].tolist()
print('\nBest Feature Subset:')
print(best_features)

rf_final = RandomForestClassifier(
    n_estimators=400, random_state=1, class_weight='balanced')
rf_final.fit(
    XF_train[best_features], yF_train)
yF_pred = rf_final.predict(
    XF_test[best_features])

print('\nFinal Model Evaluation with Best Features:')
print(f'Accuracy: {accuracy_score(yF_test, yF_pred):.4f}')
print('Classification Report:')
print(classification_report(yF_test, yF_pred))
print('Confusion Matrix:')
print(confusion_matrix(yF_test, yF_pred))

# Plot Feature Importance
importances = rf_final.feature_importances_
feat_imp = pd.DataFrame({
    'Feature': best_features,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

print('\nTop Features Driving Model Accuracy:')
display(feat_imp)

plt.figure(figsize=(8,6))
plt.barh(feat_imp['Feature'], feat_imp['Importance'], color='steelblue')
plt.gca().invert_yaxis()
plt.xlabel('Feature Importance (RF)')
plt.title('Key Demographic Predictors of Voting Patterns')
plt.show()

## Train FINAL MODEL

In [ ]:
# Select final features: From 'accuracy vs. number of features' plot, ideal number of features start at 8, 9, or 10, compare 8-10 features for accuracy and MANOVA scores (dropped HH_MARRIED due to correlation with REL_xxx_MAR)

final_features = [ # Drop '%AGE_FEMALE_YOUNG', '%REL_W_RELATIVES' for best results
    'GEOID', 'PARTY_WIN', 'PARTY_LEAD',
    '%RACE_White', '%RACE_Asian', '%Urban_pop', '%REL_S_SEX_MAR',
    '%REL_OP_SEX_MAR', '%OWN_HOME', '%REL_NON_REL', '%RACE_Black']
VOTE_FINAL = VOTE_DF[final_features]

# Set features for final model
X_final = [col for col in VOTE_FINAL.columns if col not in [
    'GEOID', 'PARTY_WIN', 'PARTY_LEAD']]
y_final = VOTE_FINAL['PARTY_WIN']

# Train/test split
XF_train, XF_test, yF_train, yF_test = train_test_split(
    VOTE_FINAL[X_final], y_final,
    test_size=0.2, random_state=1,
    stratify=y_final)

# Train Random Forest
rf_final = RandomForestClassifier(
    n_estimators=500,
    random_state=1,
    class_weight='balanced')
rf_final.fit(XF_train, yF_train)

# Evaluate
yF_pred = rf_final.predict(XF_test)

print('\nFinal Model Evaluation:')
print(f'Accuracy: {accuracy_score(yF_test, yF_pred):.4f}')
print('Classification Report:')
print(classification_report(yF_test, yF_pred))
print('Confusion Matrix:')
print(confusion_matrix(yF_test, yF_pred))

## Use MANOVA to assess whether multiple features jointly differ between Democrats and Republicans

In [ ]:
maov = MANOVA(endog=VOTE_FINAL[X_final], exog=VOTE_FINAL[[y_final.name]])
print(maov.mv_test())

## Run CV PI

In [ ]:
# Cross-validate Permutation Importance
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
importances = []

# Pass feature data to skf.split
for train_fcv, test_fcv in skf.split(VOTE_FINAL[X_final], y_final):
    XF_train, XF_test = VOTE_FINAL[X_final].iloc[train_fcv], VOTE_FINAL[X_final].iloc[test_fcv]
    yF_train, yF_test = y_final.iloc[train_fcv], y_final.iloc[test_fcv]

    rf_final.fit(XF_train, yF_train)
    result = permutation_importance(
        rf_final, XF_test, yF_test,
        n_repeats=10, random_state=1, n_jobs=-1)
    importances.append(result.importances_mean)

mean_importances = np.mean(importances, axis=0)
std_importances = np.std(importances, axis=0)

# Build PI DF
pi_df = pd.DataFrame({
    'Feature': X_final, # Use X_final for feature names
    'Importance Mean': mean_importances.round(4),
    'Importance Std': std_importances.round(4)
}).sort_values(by='Importance Mean', ascending=False)

# Confirm
print('\nCross-validated Permutation Importance:')
print(pi_df)

# Plot
plt.figure(figsize=(10, 6))
pi_df.set_index('Feature')['Importance Mean'].sort_values().plot(kind='barh')
plt.title('Final Model - CV Permutation Importance')
plt.xlabel('Mean Importance (± CV variation)')
plt.ylabel('Feature')
plt.show()

# Profile counties with **extremely high PARTY_LEAD** (-30 > LEAD > +30)  

Do the demographics of partisan counties match final feature importance?

In [ ]:
VOTE_FINAL.to_csv('VOTE_FINAL.csv', index=False)

## 1. Equal Cutoff Strongholds (±0.5)

In [ ]:
def profile_group(dataframe, name, features):
    '''Calculates the mean of specified features for a group and returns a Series.'''
# Ensure only numeric columns in features are selected for mean calculation
    numeric_features = dataframe[features].select_dtypes(include=np.number).columns.tolist()
    profile = dataframe[numeric_features].mean()
    profile.name = name
    return profile

# Set cutoff value to 0.50
cutoff_val = 0.50

# Use _FIN to allow R and D access to all variables
extreme_counties = VOTE_FULL[np.abs(VOTE_FULL['PARTY_LEAD']) > cutoff_val]

# Republican strongholds
extreme_R = extreme_counties[extreme_counties['PARTY_LEAD'] < -cutoff_val]

# Democratic strongholds
extreme_D = extreme_counties[extreme_counties['PARTY_LEAD'] > cutoff_val]

print(f'Republican strongholds (cutoff -{cutoff_val}):', extreme_R.shape[0])
print(f'Democratic strongholds (cutoff +{cutoff_val}):', extreme_D.shape[0])

# Select demographic features only (drop outcomes and GEOID)
demo_features = [col for col in VOTE_FULL.columns if col not in ['PARTY_WIN', 'PARTY_LEAD', 'GEOID']]

# Profiles for cutoff-based groups
cutoff_profiles_combined = pd.concat([
    profile_group(extreme_R, 'R_characteristics', demo_features),
    profile_group(extreme_D, 'D_characteristics', demo_features),
], axis=1)

# Add absolute difference column
cutoff_profiles_combined['Abs_Diff'] = np.abs(cutoff_profiles_combined['R_characteristics'] - cutoff_profiles_combined['D_characteristics'])

print(f'\n=== Cutoff-based Stronghold Profiles (>{cutoff_val} Party Lead) ===')
print(cutoff_profiles_combined.sort_values(by='Abs_Diff', ascending=False))

## 2. Balanced Strongholds (Top/Bottom 10% quantiles)

In [ ]:
# Set cutoff value
lower_10 = VOTE_FULL['PARTY_LEAD'].quantile(0.10)   # bottom 10% cutoff
upper_10 = VOTE_FULL['PARTY_LEAD'].quantile(0.90)   # top 10% cutoff

R_stnghd_bal = VOTE_FULL[VOTE_FULL['PARTY_LEAD'] <= lower_10].copy()
D_stnghd_bal = VOTE_FULL[VOTE_FULL['PARTY_LEAD'] >= upper_10].copy()

print(f'Republican strongholds (quantile-based): {len(R_stnghd_bal)} counties (<= {lower_10:.2f})')
print(f'Democratic strongholds (quantile-based): {len(D_stnghd_bal)} counties (>= {upper_10:.2f})')

# Select demographic features only (drop outcomes and GEOID)
demo_features = [col for col in VOTE_FULL.columns if col not in [
    'GEOID', 'PARTY_WIN', 'PARTY_LEAD']]

# Profiles for quantile-based groups
balanced_profiles = pd.concat([
    profile_group(R_stnghd_bal, 'R_characteristics', demo_features),
    profile_group(D_stnghd_bal, 'D_characteristics', demo_features),
], axis=1)

# Add absolute difference column
balanced_profiles['Abs_Diff'] = np.abs(balanced_profiles['R_characteristics'] - balanced_profiles['D_characteristics'])

print('\n=== Quantile-based Stronghold Profiles (Top/Bottom 10%) ===')
print(balanced_profiles.sort_values(by='Abs_Diff', ascending=False))

## Comparison Table

In [ ]:
# Build combined comparison table

# Profiles for cutoff-based groups
# Select demographic features only (drop outcomes and GEOID)
demo_features = [col for col in VOTE_FULL.columns if col not in ['PARTY_WIN', 'PARTY_LEAD', 'GEOID']]

cutoff_profiles = pd.concat([
    profile_group(extreme_R, 'R_cutoff', demo_features),
    profile_group(extreme_D, 'D_cutoff', demo_features)
], axis=1)

# Profiles for quantile-based groups
# Select demographic features only (drop outcomes and GEOID)
demo_features = [col for col in VOTE_FULL.columns if col not in ['PARTY_WIN', 'PARTY_LEAD', 'GEOID']]

balanced_profiles = pd.concat([
    profile_group(R_stnghd_bal, 'R_quantile', demo_features),
    profile_group(D_stnghd_bal, 'D_quantile', demo_features)
], axis=1)

# Combine both into one big table
comparison_table = pd.concat([cutoff_profiles, balanced_profiles], axis=1)

# Add difference columns (D – R) for clarity of spread
comparison_table['Diff_cutoff'] = comparison_table['D_cutoff'] - comparison_table['R_cutoff']
comparison_table['Diff_quantile'] = comparison_table['D_quantile'] - comparison_table['R_quantile']

comparison_table = comparison_table.round(2)

# Confirm
print('\n=== Combined Stronghold Profiles (Cutoff vs Quantile) ===')
display(comparison_table.sort_values(by='R_cutoff', ascending=False))

## Locations of Extremes

In [ ]:
# Pad GEOIDs with a leading zero if length is less than 5 and extract state FIPS
R_geoids = [f'{int(geo):05d}' if len(geo) < 5 else geo for geo in extreme_R['GEOID'].unique()]
D_geoids = [f'{int(geo):05d}' if len(geo) < 5 else geo for geo in extreme_D['GEOID'].unique()]

R_fips = [geo[:2] for geo in R_geoids]
D_fips = [geo[:2] for geo in D_geoids]

R_counts = Counter(R_fips)
D_counts = Counter(D_fips)

print('\nCounts for counties above 50% party lead (Min 75-25% split):')
# Sort state_counts by 2-digit state FIPS)
sorted_R_counts = dict(sorted(R_counts.items()))
sorted_D_counts = dict(sorted(D_counts.items()))
print(sorted_R_counts)
print(sorted_D_counts)

# Results

## 1. Model Performance & Prediction Quality

In [ ]:
# Predictions: Use the subset of features that the model was trained on
yF_pred = rf_final.predict(XF_test)
y_pred_proba = rf_final.predict_proba(XF_test)[:, 1] if hasattr(
               rf_final, 'predict_proba') else None

# Collect metrics
performance = {
    'Accuracy': accuracy_score(yF_test, yF_pred),
    'Precision': precision_score(yF_test, yF_pred, zero_division=0),
    'Recall': recall_score(yF_test, yF_pred, zero_division=0),
    'F1 Score': f1_score(yF_test, yF_pred, zero_division=0)}

if y_pred_proba is not None:
    performance['ROC-AUC'] = roc_auc_score(yF_test, y_pred_proba)

# Create performance DataFrame
perf_df = pd.DataFrame(performance, index=['Final Model']).T
display(perf_df)

## 2. Feature Importance

In [ ]:
print('\nCross-validated Permutation Importance:')
print(pi_df)

# Plot
plt.figure(figsize=(10, 6))
pi_df.set_index('Feature')['Importance Mean'].sort_values().plot(kind='barh')
plt.title('Final Model - CV Permutation Importance')
plt.xlabel('Mean Importance (± CV variation)')
plt.ylabel('Feature')
plt.show()

## 3. Extreme County Locations

STRONG REPUBLICAN COUNTIES-----STRONG DEMOCRAT
COUNTIES  
01: Alabama (23),------------------------------Alabama (2),  
04: Arizona (1),  
05: Arkansas(34),  
06: California (1),------------------------------California (6),  
08: Colorado (14),-----------------------------Colorado (4),  
11: ----------------------------------------------------D.C. (1),  
12: Florida (13),  
13: Georgia (40),--------------------------------Georgia (2),  
16: Idaho (27),  
17: Illinois (26),  
18: Indiana (31),  
19: Iowa (8),  
20: Kansas (67),  
21: Kentucky (69),  
22: Louisiana (15),-----------------------------Louisiana (1),  
24: Maryland (1),--------------------------------Maryland (3),  
25: -----------------------------------------------------Massachusetts (2),  
26: Michigan (1),   
27: Minnesota (1),  
28: Mississippi (15),---------------------------Mississippi (4),  
29: Missouri (78),-------------------------------Missouri (1),  
30: Montana (24),  
31: Nebraska (67),  
32: New Hampshire (8),  
34: -----------------------------------------------------New Jersey (1),  
35: New Mexico (3),---------------------------New Mexico (2),  
36: -----------------------------------------------------New York (3),  
37: North Carolina (12),----------------------North Carolina (2),  
38: North Dakota (28),  
39: Ohio (26),  
40: Oklahoma (59),  
41: Oregon (5),------------------------------------Oregon (1),  
42: Pennsylvania (11),-------------------------Pennsylvania (1),  
45: South Carolina (1),------------------------South Carolina (1),  
46: South Dakota (23),------------------------South Dakota (2),  
47: Tennessee (61),  
48: Texas (160),  
49: Utah (17),  
50: ------------------------------------------------------Vermont (1),  
51: Virginia (17),----------------------------------Virginia (6),  
53: ------------------------------------------------------Washington (2),  
54: West Virginia (31),  
55: -----------------------------------------------------Wisconsin (2),  
56: Wyoming (16)

Looking at the distribution of strongholds, the Republican base is both wide and dense, with strong states in the Deep South, the Midwest, and the Plains states—particularly Missouri, Kansas, Nebraska, Oklahoma, and Texas.  
Democrats, by contrast, have far fewer strongholds, often limited to isolated urban counties scattered within overwhelmingly Republican states. The most surprising pattern is in California: despite its reputation as a Democratic stronghold at the state level, only six counties emerged as strongly Democratic, compared to a single Republican county. Equally notable are the scattered Democratic enclaves in heavily Republican states such as Mississippi, South Dakota, and Louisiana, showing how local dynamics can carve out exceptions even in states dominated by the opposite party.  
These findings highlight that state-level reputation can sometimes mask county-level complexity in partisan alignment.

## 4. Extreme County Profiles

The comparison of extreme Republican‐leaning and Democratic‐leaning counties reveals demographic and social patterns that align closely with prior research on U.S. voting behavior.  
- Republican counties tend to be in rural areas, majority white, married, older, and more likely to own their homes. There are also somewhat higher shares of older residents living alone.

- By contrast, Democratic counties stand out for their racial and ethnic diversity, with substantially higher percentages of Black, Latino, Asian, Native, and mixed‐race residents. They are also much more urbanized, with over 70% of the population living in urban areas compared to under 20% in Republican strongholds, which leads to higher rentals and lower home ownership. Democratic counties show higher shares of unmarried households, non‐relatives living together, and female‐headed households with children. Younger age distributions also feature more prominently, with both male and female populations skewing younger than in Republican counties.

Overall, the patterns are not surprising. They mirror well‐documented demographic divides in U.S. elections: rural, older, and predominantly White populations lean Republican, while urban, younger, and racially diverse populations lean Democratic. These results validate the modeling approach, as the county‐level features align strongly with real‐world voting dynamics.



## 5. Future Steps and Broader Insights

While this analysis captures core demographic and household correlations to voting behavior, it omits several dimensions that could strengthen explanatory power and refine predictions. Socioeconomic variables—such as income, education levels, and employment sectors—are known to shape political preferences and could highlight additional divides within and across counties. Migration trends, naturalization status, and geographic mobility could also explain differences in partisan lean, particularly in fast‐growing metro regions.

In addition, county‐level aggregates obscure within‐county variation, especially in large metropolitan areas where neighborhoods diverge sharply in demographics and partisanship. Incorporating finer spatial resolution (e.g., census tract or voter precinct) or longitudinal trends over multiple election cycles could help reveal whether these patterns are persistent or shifting. Finally, the integration of turnout variables—distinguishing who is registered, eligible, and actually voting—would broaden the analysis beyond demographic composition to electoral engagement.

Together, these additions would not only deepen the descriptive accuracy but also broaden the explanatory scope of the findings, linking demographic patterns more directly with political outcomes.

END